In [14]:
%matplotlib widget
import math
import random
import sys
from kan import *
from kan.utils import create_dataset
from kan.utils import ex_round
torch.set_default_dtype(torch.float32)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Parameters
input_size = 1
hidden_layers = 5#10
output_size = 1
epochs = 50
learning_rate = 0.001
variance = 0.02
mean = 0
train_samples = 1000
test_samples = 1000
interval = (0, 2*torch.pi)

# Number of tests
iterations = 1000

cpu


In [2]:
f = lambda x: torch.sin(x[:,[0]]) # + x[:,[0]]**2
# Use torch.linspace for direct PyTorch tensor creation
x = torch.linspace(interval[0], interval[1], train_samples)
x = torch.stack([x.ravel()], dim=-1)
y_clear = f(x)
dataset = create_dataset(
    f, 
    n_var=1, 
    device=device, 
    normalize_input=False, 
    normalize_label=False, 
    train_num=train_samples, 
    test_num=test_samples,
    ranges=[interval[0], interval[1]],
)
#lib = ['x','x^2','x^3','x^4','exp','log','sqrt','tanh','sin','abs']
lib = ['x','x^2','x^3','x^4','exp','log','sqrt','tanh','sin']

def add_white_noise(x_train, mean=0.0, variance=0.01):
    """
    Adds white noise to the input data.

    Args:
        x_train (torch.Tensor): The input training data (e.g., x_train).
        mean (float): Mean of the Gaussian noise to be added.
        variance (float): Variance of the Gaussian noise to be added.

    Returns:
        torch.Tensor: The noisy data.
    """
    # Generate Gaussian noise (mean, variance) with the same shape as x_train
    random.seed(None)
    random_seed = random.randint(-sys.maxsize, sys.maxsize)
    torch.manual_seed(random_seed)
    noise = torch.randn(x_train.size()) * torch.sqrt(torch.tensor(variance)) + mean
    #noise = torch.normal(mean=mean, std=torch.sqrt(torch.tensor(variance)), size=x_train.size())
    
    # Add noise to the original training data
    noisy_data = x_train + noise

    return noisy_data

def iteration_callback():

    # Use torch.linspace for direct PyTorch tensor creation
    y = add_white_noise(y_clear, 0.0, variance)
    dataset['train_input'] = x
    dataset['train_label'] = y

    # create a KAN: 2D inputs, 1D output, and 5 hidden neurons. cubic spline (k=3), 5 grid intervals (grid=5).
    kan_model = MultKAN(
        width=[input_size, hidden_layers, hidden_layers, output_size],
        grid=5, 
        k=3,
        #seed=42, 
        device=device, 
        #grid_range= [interval[0], interval[1]],
        #noise_scale=0.0,
        #affine_trainable=True,
        #grid_eps=0,
        auto_save=False,
    )

    kan_model.fit(
        dataset, 
        opt = "LBFGS", 
        steps = epochs,
        update_grid = True,
        lamb = learning_rate,
        #lamb_l1 = 100,
        #lamb_entropy = 100,
        #lamb_coef = 100,
        #lamb_coefdiff = 100,
        loss_fn = torch.nn.MSELoss(),
        log=-1,
    )

    kan_model = kan_model.prune(
        node_th = 1e-1
        #node_th = 0.02,
        #edge_th = 0.06
    )
    kan_model.fit(
        dataset, 
        opt = "LBFGS", 
        steps = epochs,
        update_grid = True,
        lamb = learning_rate/2,
        #lamb_l1 = 100,
        #lamb_entropy = 100,
        #lamb_coef = 100,
        #lamb_coefdiff = 100,
        loss_fn = torch.nn.MSELoss(),
        log=-1,
    )

    kan_model.auto_symbolic(
        lib=lib,
        #a_range=(-1,1),
        #b_range=(-1,1),
        r2_threshold=0.9,
        verbose=0,
    )
    kan_model.fit(
        dataset, 
        opt = "LBFGS", 
        steps = epochs,
        update_grid = True,
        lamb = learning_rate,
        #lamb_l1 = 100,
        #lamb_entropy = 100,
        #lamb_coef = 100,
        #lamb_coefdiff = 100,
        loss_fn = torch.nn.MSELoss(),
        log=-1,
    )

    training_data_codomain = (float(min(y)[0]), float(max(y)[0]))
    kan_model_output = kan_model(dataset['train_input']).detach().cpu().numpy()
    kan_model_codomain = (float(min(kan_model_output)[0]), float(max(kan_model_output)[0]))
    equation = ""#ex_round(kan_model.symbolic_formula()[0][0], 4)

    return training_data_codomain, kan_model_codomain, equation

In [ ]:
print(f"| Iteration | Data Codomain | KAN Codomain | KAN Equation |")
print(f"|-----------|---------------|--------------|--------------|")
i = 0
while i < iterations:
    try:
        training_data_codomain, kan_model_codomain, equation = iteration_callback()
        output = f"| {i+1} | {training_data_codomain} | {kan_model_codomain} | {equation} |"
        if not ("nan" in output):
            print(output)
            i += 1
    except:
        pass

| Iteration | Data Codomain | KAN Codomain | KAN Equation |
|-----------|---------------|--------------|--------------|
| 1 | (-1.4870350360870361, 1.2926733493804932) | (-0.9883950352668762, 1.0056040287017822) |  |
| 2 | (-1.2902319431304932, 1.4405460357666016) | (-1.0128260850906372, 1.0111218690872192) |  |
| 3 | (-1.3410931825637817, 1.2736294269561768) | (-1.0059422254562378, 0.9799834489822388) |  |
| 4 | (-1.4774134159088135, 1.2854126691818237) | (-1.006138563156128, 1.0041884183883667) |  |
| 5 | (-1.4497206211090088, 1.3384045362472534) | (-0.9853233695030212, 1.0070114135742188) |  |
| 6 | (-1.2639354467391968, 1.3627811670303345) | (-0.9963324666023254, 1.0001823902130127) |  |
| 7 | (-1.295318365097046, 1.290456771850586) | (-0.9637668132781982, 0.9940399527549744) |  |
| 8 | (-1.374756097793579, 1.312765121459961) | (-1.0060169696807861, 1.0007156133651733) |  |
| 9 | (-1.3059074878692627, 1.3331844806671143) | (-0.9975211024284363, 1.0201722383499146) |  |
| 10 | (-1.3

/Users/albertopetrucci/Documents/github/pykan/kan/KANLayer.py:110: RuntimeWarning: invalid value encountered in scalar divide
  self.scale_base = torch.nn.Parameter(scale_base_mu * 1 / np.sqrt(in_dim) + \


| 221 | (-1.3040776252746582, 1.3181984424591064) | (-0.9053628444671631, 1.0548311471939087) |  |
| 222 | (-1.3959732055664062, 1.3126529455184937) | (nan, nan) |  |
| 223 | (-1.4052133560180664, 1.3057639598846436) | (-1.0106050968170166, 1.0170515775680542) |  |
| 224 | (-1.4045178890228271, 1.3439565896987915) | (-1.0101516246795654, 1.0016772747039795) |  |
| 225 | (-1.285249948501587, 1.3140954971313477) | (-0.9929457306861877, 1.0161826610565186) |  |
| 226 | (-1.5067739486694336, 1.4287724494934082) | (-0.9424609541893005, 1.0399516820907593) |  |
| 227 | (-1.2958667278289795, 1.2416387796401978) | (-1.0108364820480347, 1.0049073696136475) |  |
| 228 | (-1.3508884906768799, 1.3625483512878418) | (-1.0053447484970093, 0.9961518049240112) |  |
| 229 | (-1.3664581775665283, 1.3643471002578735) | (-0.9997828602790833, 0.9943053722381592) |  |
| 230 | (-1.3538224697113037, 1.4813568592071533) | (-0.99784916639328, 1.0060646533966064) |  |
| 231 | (-1.4212148189544678, 1.340918421745

/Users/albertopetrucci/Documents/github/pykan/kan/KANLayer.py:110: RuntimeWarning: invalid value encountered in scalar divide
  self.scale_base = torch.nn.Parameter(scale_base_mu * 1 / np.sqrt(in_dim) + \


| 346 | (-1.324379324913025, 1.4580700397491455) | (-1.0026434659957886, 1.0090621709823608) |  |
| 347 | (-1.3639869689941406, 1.3224987983703613) | (-0.9966335296630859, 0.9884900450706482) |  |
| 348 | (-1.29929780960083, 1.3388253450393677) | (-1.0006657838821411, 1.0137555599212646) |  |
| 349 | (-1.2510915994644165, 1.380838394165039) | (-1.0076205730438232, 1.0027977228164673) |  |
| 350 | (-1.3531687259674072, 1.349729299545288) | (-0.9868287444114685, 0.9944071173667908) |  |
| 351 | (-1.3651255369186401, 1.3383623361587524) | (-1.009272813796997, 1.000351071357727) |  |
| 352 | (-1.3720674514770508, 1.4521996974945068) | (-0.9956598877906799, 0.997035801410675) |  |
| 353 | (-1.308140516281128, 1.2787492275238037) | (-1.0195940732955933, 1.006447434425354) |  |
| 354 | (-1.2367993593215942, 1.4482356309890747) | (-0.9859819412231445, 0.9942251443862915) |  |
| 355 | (-1.3676663637161255, 1.3192706108093262) | (-0.9935685992240906, 0.9954193234443665) |  |
| 356 | (-1.35209441

/Users/albertopetrucci/Documents/github/pykan/kan/KANLayer.py:110: RuntimeWarning: invalid value encountered in scalar divide
  self.scale_base = torch.nn.Parameter(scale_base_mu * 1 / np.sqrt(in_dim) + \


| 498 | (-1.3655529022216797, 1.3086752891540527) | (-1.001844048500061, 0.9916096329689026) |  |
| 499 | (-1.2978917360305786, 1.3443915843963623) | (-1.0058064460754395, 1.0124694108963013) |  |
| 500 | (-1.34653902053833, 1.3598421812057495) | (-0.9984822869300842, 0.9984436631202698) |  |
| 501 | (-1.372138500213623, 1.442349910736084) | (-0.9959689378738403, 0.990826427936554) |  |
| 502 | (-1.3351343870162964, 1.3476024866104126) | (-0.9585639834403992, 0.9900230169296265) |  |
| 503 | (-1.3993667364120483, 1.3274673223495483) | (-1.003829836845398, 0.9847605228424072) |  |
| 504 | (-1.2283474206924438, 1.4144750833511353) | (-0.9517577290534973, 1.0233572721481323) |  |
| 505 | (-1.2997276782989502, 1.415611982345581) | (-0.9932822585105896, 1.0107536315917969) |  |
| 506 | (-1.3253679275512695, 1.3576549291610718) | (-1.006799340248108, 1.007951259613037) |  |
| 507 | (-1.3192545175552368, 1.3700947761535645) | (-0.9391422867774963, 1.0091685056686401) |  |
| 508 | (-1.34936308

In [30]:
from IPython.display import display
def parse_table(text):
    lines = text.strip().split("\n")
    data = []
    
    for line in lines[2:]:  # Skip the header lines
        match = re.match(r'\|\s*(\d+)\s*\|\s*\(([^,]+), ([^\)]+)\)\s*\|\s*\(([^,]+), ([^\)]+)\)\s*\|', line)
        if match:
            iteration = int(match.group(1))
            data_codomain = (float(match.group(2)), float(match.group(3)))
            kan_codomain = (float(match.group(4)), float(match.group(5)))
            
            data.append({
                "Iteration": iteration,
                "Data Codomain": data_codomain,
                "KAN Codomain": kan_codomain
            })
    
    return data

def performance_statistics(parsed_data, key, mean, variance, real_codomain=(-1, 1)):
    codomains = []
    deviations = []
    for entry in parsed_data:
        x, y = entry[key]
        codomains.append((x, y))
        deviation_x = abs(x - real_codomain[0])
        deviation_y = abs(y - real_codomain[1])
        deviations.append((deviation_x, deviation_y))
    
    codomains = np.array(codomains)
    deviations = np.array(deviations)
    mean_codomain = np.mean(codomains, axis=0)
    mean_deviation = np.mean(deviations, axis=0)
    max_deviation = np.max(deviations, axis=0)
    min_deviation = np.min(deviations, axis=0)

    below_variance_x = np.sum(deviations[:, 0] <= abs(real_codomain[0]*variance)) / len(deviations) * 100
    below_variance_y = np.sum(deviations[:, 1] <= abs(real_codomain[1]*variance)) / len(deviations) * 100
    
    stats = {
        "Real Codomain": real_codomain,
        "Mean Codomain": mean_codomain,
        "Mean Deviation": mean_deviation,
        "Max Deviation": max_deviation,
        "Min Deviation": min_deviation,
        "Bounding Variance": [real_codomain[0]*variance, real_codomain[1]*variance],
        "% Below Variance": [below_variance_x, below_variance_y]
    }

    df = pd.DataFrame(stats, index=["X", "Y"])
    display(df)
    return df

text = """| Iteration | Data Codomain | KAN Codomain |
|-----------|---------------|--------------|
| 1 | (-1.4870350360870361, 1.2926733493804932) | (-0.9883950352668762, 1.0056040287017822) |
| 2 | (-1.2902319431304932, 1.4405460357666016) | (-1.0128260850906372, 1.0111218690872192) |
| 3 | (-1.3410931825637817, 1.2736294269561768) | (-1.0059422254562378, 0.9799834489822388) |
| 4 | (-1.4774134159088135, 1.2854126691818237) | (-1.006138563156128, 1.0041884183883667) |
| 5 | (-1.4497206211090088, 1.3384045362472534) | (-0.9853233695030212, 1.0070114135742188) |
| 6 | (-1.2639354467391968, 1.3627811670303345) | (-0.9963324666023254, 1.0001823902130127) |
| 7 | (-1.295318365097046, 1.290456771850586) | (-0.9637668132781982, 0.9940399527549744) |
| 8 | (-1.374756097793579, 1.312765121459961) | (-1.0060169696807861, 1.0007156133651733) |
| 9 | (-1.3059074878692627, 1.3331844806671143) | (-0.9975211024284363, 1.0201722383499146) |
| 10 | (-1.3216098546981812, 1.3525649309158325) | (-0.9848595261573792, 0.9686763286590576) |
| 11 | (-1.3945825099945068, 1.2993476390838623) | (-1.0089823007583618, 1.0152798891067505) |
| 12 | (-1.3755626678466797, 1.4067530632019043) | (-0.9996148347854614, 0.9894776344299316) |
| 13 | (-1.3332439661026, 1.288145661354065) | (-1.0150716304779053, 0.9957585334777832) |
| 14 | (-1.2938385009765625, 1.263792872428894) | (-0.9765943288803101, 0.9720515608787537) |
| 15 | (-1.3658623695373535, 1.350871205329895) | (-0.9790768027305603, 0.9996190667152405) |
| 16 | (-1.2731056213378906, 1.363408088684082) | (-1.002440094947815, 0.9999629259109497) |
| 17 | (-1.2573728561401367, 1.3702178001403809) | (-0.9932382106781006, 0.9955871105194092) |
| 18 | (-1.3307281732559204, 1.3570183515548706) | (-1.007565975189209, 1.009203314781189) |
| 19 | (-1.300455093383789, 1.4025262594223022) | (-0.9973890781402588, 1.0124330520629883) |
| 20 | (-1.3129231929779053, 1.295491337776184) | (-0.9907893538475037, 0.9974775314331055) |
| 21 | (-1.328675627708435, 1.398059606552124) | (-1.0048068761825562, 0.9989836812019348) |
| 22 | (-1.3236112594604492, 1.3314615488052368) | (-0.9988158941268921, 0.9952433109283447) |
| 23 | (-1.3413822650909424, 1.3197596073150635) | (-0.9886003732681274, 0.9406316876411438) |
| 24 | (-1.310451865196228, 1.337315320968628) | (-1.0070325136184692, 1.0021815299987793) |
| 25 | (-1.36360764503479, 1.3660173416137695) | (-1.0096147060394287, 1.0187610387802124) |
| 26 | (-1.3984980583190918, 1.3255646228790283) | (-1.0510549545288086, 1.0992263555526733) |
| 27 | (-1.2789613008499146, 1.3778293132781982) | (-0.9889825582504272, 0.9911974668502808) |
| 28 | (-1.3857836723327637, 1.41672945022583) | (-1.004223108291626, 0.9917687177658081) |
| 29 | (-1.35024094581604, 1.3966788053512573) | (-1.0021926164627075, 0.9991840720176697) |
| 30 | (-1.301195740699768, 1.272769570350647) | (-0.9914422035217285, 1.0032082796096802) |
| 31 | (-1.4919497966766357, 1.4120099544525146) | (-0.9994792938232422, 1.0062724351882935) |
| 32 | (-1.3031246662139893, 1.2904949188232422) | (-1.0127694606781006, 0.9981248378753662) |
| 33 | (-1.3152942657470703, 1.2650916576385498) | (-1.0090824365615845, 0.9947004318237305) |
| 34 | (-1.314022421836853, 1.2754415273666382) | (-0.9922393560409546, 0.9965951442718506) |
| 35 | (-1.2354991436004639, 1.26517915725708) | (-1.0002131462097168, 0.9919648766517639) |
| 36 | (-1.3390262126922607, 1.3495256900787354) | (-1.0020952224731445, 0.9988770484924316) |
| 37 | (-1.3287830352783203, 1.3545944690704346) | (-0.8606054186820984, 1.054028868675232) |
| 38 | (-1.3302443027496338, 1.3655465841293335) | (-0.9922818541526794, 1.0007903575897217) |
| 39 | (-1.314967155456543, 1.2960073947906494) | (-0.9907035827636719, 0.9925794005393982) |
| 40 | (-1.2441601753234863, 1.3977038860321045) | (-1.0163334608078003, 1.001738429069519) |
| 41 | (-1.2774029970169067, 1.418372392654419) | (-0.9693048000335693, 0.9840965867042542) |
| 42 | (-1.3184077739715576, 1.348966121673584) | (-1.0086535215377808, 1.0034775733947754) |
| 43 | (-1.3393762111663818, 1.289396047592163) | (-1.0098260641098022, 1.003801941871643) |
| 44 | (-1.315978765487671, 1.3197377920150757) | (-1.0098261641098022, 1.003800941871643) |
| 45 | (-1.3204829692840576, 1.2992162704467773) | (-1.0098260541098022, 1.00380841871643) |
| 46 | (-1.3793702125549316, 1.3605279922485352) | (-1.0052495002746582, 0.9990248680114746) |
| 47 | (-1.281523585319519, 1.3470203876495361) | (-0.9911989569664001, 0.9944191575050354) |
| 48 | (-1.335747241973877, 1.3139564990997314) | (-0.9948088526725769, 1.0193824768066406) |
| 49 | (-1.357378363609314, 1.3874318599700928) | (-1.0046221017837524, 0.9951802492141724) |
| 50 | (-1.492112398147583, 1.348275065422058) | (-1.005111575126648, 1.0081027746200562) |
| 51 | (-1.2460390329360962, 1.2994722127914429) | (-0.9979580044746399, 1.0036150217056274) |
| 52 | (-1.329378604888916, 1.5763404369354248) | (-1.0100793838500977, 1.0034383535385132) |
| 53 | (-1.3337624073028564, 1.4092851877212524) | (-0.9464709162712097, 1.0572824478149414) |
| 54 | (-1.3667207956314087, 1.291347622871399) | (-0.998143196105957, 0.9976152181625366) |
| 55 | (-1.3254927396774292, 1.3578723669052124) | (-0.9910380244255066, 1.0158864259719849) |
| 56 | (-1.291355013847351, 1.3613665103912354) | (-0.9876869320869446, 1.0067603588104248) |
| 57 | (-1.4045805931091309, 1.3503408432006836) | (-0.9774582386016846, 1.0062530040740967) |
| 58 | (-1.4552836418151855, 1.3567568063735962) | (-0.9799444675445557, 0.9849430322647095) |
| 59 | (-1.3359824419021606, 1.3130395412445068) | (-0.9814876317977905, 0.9788296818733215) |
| 60 | (-1.3677520751953125, 1.3343549966812134) | (-0.9831956028938293, 1.008856177330017) |
| 61 | (-1.3909406661987305, 1.380611538887024) | (-1.0104429721832275, 1.0038988590240479) |
| 62 | (-1.3696575164794922, 1.3085781335830688) | (-0.989181399345398, 1.0009599924087524) |
| 63 | (-1.3486237525939941, 1.3083627223968506) | (-0.9981383085250854, 0.9973770380020142) |
| 64 | (-1.2768374681472778, 1.3724771738052368) | (-0.9953165650367737, 0.9938662052154541) |
| 65 | (-1.4510304927825928, 1.3555070161819458) | (-1.003015160560608, 0.9910466074943542) |
| 66 | (-1.2892346382141113, 1.3461886644363403) | (-0.9945759177207947, 0.9955177307128906) |
| 67 | (-1.4140957593917847, 1.3395347595214844) | (-1.0074536800384521, 1.006223440170288) |
| 68 | (-1.4146313667297363, 1.2929717302322388) | (-1.0135250091552734, 0.9921036958694458) |
| 69 | (-1.330647349357605, 1.3489551544189453) | (-0.9553931951522827, 0.9495297074317932) |
| 70 | (-1.2932153940200806, 1.3985176086425781) | (-0.8994385004043579, 1.0381795167922974) |
| 71 | (-1.3580312728881836, 1.2829632759094238) | (-0.9900850057601929, 0.9923118352890015) |
| 72 | (-1.3621189594268799, 1.3465343713760376) | (-0.9922648668289185, 0.9973870515823364) |
| 73 | (-1.3373674154281616, 1.2776625156402588) | (-0.9987905621528625, 0.9979991912841797) |
| 74 | (-1.3559134006500244, 1.249567985534668) | (-1.0025850534439087, 0.9970331788063049) |
| 75 | (-1.3274831771850586, 1.4261635541915894) | (-0.994766354560852, 0.9951372742652893) |
| 76 | (-1.3429301977157593, 1.262026071548462) | (-0.8925389647483826, 1.0505942106246948) |
| 77 | (-1.4360443353652954, 1.3561394214630127) | (-0.9932148456573486, 1.0001391172409058) |
| 78 | (-1.4020222425460815, 1.3154889345169067) | (-0.9939171671867371, 0.9956850409507751) |
| 79 | (-1.3322701454162598, 1.383225917816162) | (-1.0015453100204468, 1.0079123973846436) |
| 80 | (-1.2919793128967285, 1.2944053411483765) | (-1.012629747390747, 1.005958080291748) |
| 81 | (-1.3053538799285889, 1.2781952619552612) | (-1.0276719331741333, 1.02029550075531) |
| 82 | (-1.3812408447265625, 1.5521166324615479) | (-0.9874205589294434, 0.9995440244674683) |
| 83 | (-1.4251530170440674, 1.328104019165039) | (-1.007297158241272, 1.0072031021118164) |
| 84 | (-1.2817081212997437, 1.3357672691345215) | (-1.005021572113037, 0.9886376261711121) |
| 85 | (-1.2869055271148682, 1.2750524282455444) | (-0.95578932762146, 1.032869577407837) |
| 86 | (-1.323178768157959, 1.2642943859100342) | (-1.0098261541098022, 1.00380851871643) |
| 87 | (-1.2740341424942017, 1.3363200426101685) | (-0.9825265407562256, 0.9885520935058594) |
| 88 | (-1.3278151750564575, 1.3297427892684937) | (-1.005021333694458, 0.9910479784011841) |
| 89 | (-1.3781160116195679, 1.4052656888961792) | (-1.0065709352493286, 1.0002729892730713) |
| 90 | (-1.2937566041946411, 1.3528558015823364) | (-1.0126702785491943, 1.0148329734802246) |
| 91 | (-1.291918396949768, 1.4609547853469849) | (-0.9922958016395569, 1.0001683235168457) |
| 92 | (-1.3587288856506348, 1.2935537099838257) | (-0.9864064455032349, 1.023646354675293) |
| 93 | (-1.355567455291748, 1.2593358755111694) | (-1.0107667446136475, 0.9923247694969177) |
| 94 | (-1.3904799222946167, 1.372746229171753) | (-0.9941580295562744, 1.0001273155212402) |
| 95 | (-1.3254934549331665, 1.3181025981903076) | (-0.9693595170974731, 1.018350601196289) |
| 96 | (-1.3902604579925537, 1.3945307731628418) | (-0.9990075826644897, 0.999922513961792) |
| 97 | (-1.3976335525512695, 1.2847462892532349) | (-0.9967777132987976, 0.9902541041374207) |
| 98 | (-1.3622794151306152, 1.3330285549163818) | (-0.995651364326477, 0.9998100996017456) |
| 99 | (-1.3066151142120361, 1.3527051210403442) | (-1.0082404613494873, 0.9651825428009033) |
| 100 | (-1.3371303081512451, 1.3551729917526245) | (-0.9948837161064148, 1.0107996463775635) |
| 101 | (-1.4068543910980225, 1.3500862121582031) | (-1.0099530220031738, 0.9931665062904358) |
| 102 | (-1.390160322189331, 1.2738231420516968) | (-1.0039079189300537, 1.0067185163497925) |
| 103 | (-1.3520005941390991, 1.3671494722366333) | (-0.9972124099731445, 0.9963311553001404) |
| 104 | (-1.353320837020874, 1.363266944885254) | (-1.0076260566711426, 1.0052107572555542) |
| 105 | (-1.326831579208374, 1.4164198637008667) | (-0.9948643445968628, 0.9880343675613403) |
| 106 | (-1.2973421812057495, 1.3632216453552246) | (-0.9561970233917236, 0.9860676527023315) |
| 107 | (-1.2940565347671509, 1.342637062072754) | (-0.9957783818244934, 1.0018234252929688) |
| 108 | (-1.3597146272659302, 1.2925004959106445) | (-0.9960722923278809, 1.004819393157959) |
| 109 | (-1.4035308361053467, 1.4533755779266357) | (-1.0098038911819458, 1.005212664604187) |
| 110 | (-1.38275146484375, 1.332412838935852) | (-0.9903163313865662, 0.9891874194145203) |
| 111 | (-1.3340436220169067, 1.3037910461425781) | (-1.006203293800354, 1.000607967376709) |
| 112 | (-1.3075693845748901, 1.3509840965270996) | (-0.9961235523223877, 1.003691554069519) |
| 113 | (-1.2683182954788208, 1.4852409362792969) | (-0.9924494028091431, 0.9936255216598511) |
| 114 | (-1.369933843612671, 1.409032940864563) | (-1.0026214122772217, 0.9965945482254028) |
| 115 | (-1.3999031782150269, 1.4011598825454712) | (-0.9963199496269226, 0.9931764602661133) |
| 116 | (-1.3618426322937012, 1.2822418212890625) | (-0.9997584223747253, 0.9972388744354248) |
| 117 | (-1.3850921392440796, 1.3658761978149414) | (-1.0032951831817627, 1.012823462486267) |
| 118 | (-1.2674527168273926, 1.4027888774871826) | (-0.9780566096305847, 0.9982078075408936) |
| 119 | (-1.395447015762329, 1.302147626876831) | (-1.0001076459884644, 0.9834144115447998) |
| 120 | (-1.4421732425689697, 1.2984883785247803) | (-0.989683210849762, 1.0023623704910278) |
| 121 | (-1.4634079933166504, 1.3551099300384521) | (-1.0054965019226074, 0.9960924983024597) |
| 122 | (-1.2960749864578247, 1.2349534034729004) | (-1.0060094594955444, 1.0081137418746948) |
| 123 | (-1.308730125427246, 1.3725939989089966) | (-0.9799677729606628, 1.0090312957763672) |
| 124 | (-1.4890055656433105, 1.4367806911468506) | (-0.9945846796035767, 1.0036978721618652) |
| 125 | (-1.4183017015457153, 1.3082863092422485) | (-0.9986050128936768, 1.000124216079712) |
| 126 | (-1.2874314785003662, 1.2888776063919067) | (-1.0051548480987549, 0.9790741205215454) |
| 127 | (-1.2621612548828125, 1.2948778867721558) | (-0.9950380325317383, 1.0004398822784424) |
| 128 | (-1.421974539756775, 1.2948224544525146) | (-1.007042407989502, 0.9830951690673828) |
| 129 | (-1.3287602663040161, 1.4651272296905518) | (-1.0037120580673218, 0.9922271966934204) |
| 130 | (-1.4124820232391357, 1.3202190399169922) | (-0.9300848245620728, 1.028651475906372) |
| 131 | (-1.215928077697754, 1.305079460144043) | (-0.9842283725738525, 1.055094599723816) |
| 132 | (-1.3986804485321045, 1.4408708810806274) | (-0.9973902106285095, 0.9980915784835815) |
| 133 | (-1.3898266553878784, 1.2980725765228271) | (-0.994738757610321, 0.9969456791877747) |
| 134 | (-1.3138582706451416, 1.331888198852539) | (-0.9752428531646729, 1.0371575355529785) |
| 135 | (-1.4201580286026, 1.437192440032959) | (-1.0098262541098022, 1.00380852871643) |
| 136 | (-1.380734920501709, 1.4388103485107422) | (-0.9958347082138062, 0.998202919960022) |
| 137 | (-1.343252182006836, 1.3831641674041748) | (-1.0110509395599365, 0.9957634806632996) |
| 138 | (-1.2924859523773193, 1.2838566303253174) | (-0.9872264266014099, 1.0029526948928833) |
| 139 | (-1.3305413722991943, 1.256349802017212) | (-0.9931814074516296, 0.9982582926750183) |
| 140 | (-1.3492372035980225, 1.303467035293579) | (-1.0098252541098022, 1.00381852871643) |
| 141 | (-1.2984098196029663, 1.3353586196899414) | (-0.9935207366943359, 1.003772258758545) |
| 142 | (-1.3580896854400635, 1.36063551902771) | (-1.0029869079589844, 0.9932823181152344) |
| 143 | (-1.3707722425460815, 1.3711365461349487) | (-0.9922012090682983, 0.9933496117591858) |
| 144 | (-1.2963882684707642, 1.4784163236618042) | (-1.0089695453643799, 0.9992421865463257) |
| 145 | (-1.3050270080566406, 1.3418551683425903) | (-0.9624560475349426, 0.9929327964782715) |
| 146 | (-1.3044085502624512, 1.3048291206359863) | (-1.00950288772583, 0.995999276638031) |
| 147 | (-1.2591863870620728, 1.4261045455932617) | (-0.9989777207374573, 1.0060023069381714) |
| 148 | (-1.2505611181259155, 1.4028981924057007) | (-0.9957432746887207, 1.0029219388961792) |
| 149 | (-1.2910364866256714, 1.3147910833358765) | (-1.0033773183822632, 1.0024194717407227) |
| 150 | (-1.2524843215942383, 1.3490651845932007) | (-0.9939805865287781, 1.0068976879119873) |
| 151 | (-1.2652944326400757, 1.4337849617004395) | (-0.9940192699432373, 0.9882514476776123) |
| 152 | (-1.3120096921920776, 1.409921646118164) | (-1.0121121406555176, 1.0240956544876099) |
| 153 | (-1.3264822959899902, 1.3484132289886475) | (-0.9680772423744202, 1.004485845565796) |
| 154 | (-1.288562297821045, 1.2640926837921143) | (-1.0178738832473755, 1.001522421836853) |
| 155 | (-1.335040807723999, 1.35796320438385) | (-0.9619138240814209, 1.0275973081588745) |
| 156 | (-1.3376951217651367, 1.3471583127975464) | (-1.0015289783477783, 0.9842124581336975) |
| 157 | (-1.3522697687149048, 1.3445242643356323) | (-1.003672480583191, 0.9945060014724731) |
| 158 | (-1.321725845336914, 1.3292388916015625) | (-1.011826992034912, 1.0166840553283691) |
| 159 | (-1.3666404485702515, 1.44564688205719) | (-1.0002464056015015, 1.00863516330719) |
| 160 | (-1.372740626335144, 1.261147379875183) | (-1.0079807043075562, 0.9962204098701477) |
| 161 | (-1.4148175716400146, 1.3932987451553345) | (-0.9917449951171875, 1.0128415822982788) |
| 162 | (-1.3489394187927246, 1.3318605422973633) | (-1.0034185647964478, 0.9873433113098145) |
| 163 | (-1.2450242042541504, 1.3314313888549805) | (-0.9784998297691345, 1.0033823251724243) |
| 164 | (-1.3086693286895752, 1.4223012924194336) | (-0.9998368620872498, 1.0191282033920288) |
| 165 | (-1.40910804271698, 1.3358922004699707) | (-0.9899594783782959, 0.992800772190094) |
| 166 | (-1.3822882175445557, 1.435449481010437) | (-0.9960891604423523, 1.0039111375808716) |
| 167 | (-1.3791513442993164, 1.3478062152862549) | (-1.003787636756897, 1.0013453960418701) |
| 168 | (-1.3463008403778076, 1.3281575441360474) | (-1.0084257125854492, 1.0133613348007202) |
| 169 | (-1.3098056316375732, 1.382327914237976) | (-0.9947211742401123, 1.0045435428619385) |
| 170 | (-1.3350237607955933, 1.4449948072433472) | (-1.0123599767684937, 1.0202829837799072) |
| 171 | (-1.3320715427398682, 1.3564598560333252) | (-0.9118238091468811, 1.0358725786209106) |
| 172 | (-1.3964236974716187, 1.4121414422988892) | (-1.0098252541097022, 1.00381852871743) |
| 173 | (-1.318439245223999, 1.3301702737808228) | (-0.9876216053962708, 0.9861822128295898) |
| 174 | (-1.358490228652954, 1.4017837047576904) | (-1.0060402154922485, 1.0014511346817017) |
| 175 | (-1.3861421346664429, 1.3758196830749512) | (-0.9929466843605042, 0.9997135996818542) |
| 176 | (-1.3109889030456543, 1.584404706954956) | (-0.9613281488418579, 0.9840801954269409) |
| 177 | (-1.2819876670837402, 1.3907870054244995) | (-0.9919531941413879, 0.9982901215553284) |
| 178 | (-1.2458704710006714, 1.4035812616348267) | (-0.998062252998352, 1.000796914100647) |
| 179 | (-1.318240761756897, 1.328009009361267) | (-1.0079455375671387, 0.9987637400627136) |
| 180 | (-1.4745445251464844, 1.4243643283843994) | (-0.994755208492279, 0.9948012232780457) |
| 181 | (-1.2697744369506836, 1.354146122932434) | (-0.9989073276519775, 0.9964533448219299) |
| 182 | (-1.3954541683197021, 1.3761169910430908) | (-0.993889570236206, 0.9926738739013672) |
| 183 | (-1.364331841468811, 1.3267425298690796) | (-0.9970308542251587, 0.9925033450126648) |
| 184 | (-1.383981466293335, 1.376112937927246) | (-1.01217520236969, 1.0062122344970703) |
| 185 | (-1.3481277227401733, 1.356213092803955) | (-0.9800388216972351, 1.0142894983291626) |
| 186 | (-1.3914687633514404, 1.475473403930664) | (-0.9943602085113525, 1.0030882358551025) |
| 187 | (-1.3652892112731934, 1.3820202350616455) | (-0.9209633469581604, 1.045366644859314) |
| 188 | (-1.3747183084487915, 1.337098479270935) | (-1.0042895078659058, 1.002490520477295) |
| 189 | (-1.3330384492874146, 1.3241097927093506) | (-1.008508324623108, 1.0023068189620972) |
| 190 | (-1.2636412382125854, 1.225590467453003) | (-0.9980745315551758, 0.992333173751831) |
| 191 | (-1.3463122844696045, 1.381371259689331) | (-0.9963597655296326, 1.0133373737335205) |
| 192 | (-1.298052191734314, 1.3154723644256592) | (-1.0029857158660889, 1.000773310661316) |
| 193 | (-1.3803298473358154, 1.3092560768127441) | (-0.9973827004432678, 1.0009996891021729) |
| 194 | (-1.3715155124664307, 1.4288634061813354) | (-1.0021425485610962, 0.9966645240783691) |
| 195 | (-1.3377771377563477, 1.2882614135742188) | (-0.9876413345336914, 0.9923499226570129) |
| 196 | (-1.3319180011749268, 1.3885101079940796) | (-0.9996891021728516, 1.0006119012832642) |
| 197 | (-1.3631072044372559, 1.4366785287857056) | (-0.9954261779785156, 0.9974514842033386) |
| 198 | (-1.271875023841858, 1.4881372451782227) | (-0.9874984622001648, 0.9993269443511963) |
| 199 | (-1.3059351444244385, 1.3146415948867798) | (-0.9941843152046204, 1.000438928604126) |
| 200 | (-1.2910126447677612, 1.3467336893081665) | (-0.9845313429832458, 0.999597430229187) |
| 201 | (-1.466214895248413, 1.2884273529052734) | (-0.959551215171814, 1.0101710557937622) |
| 202 | (-1.217916488647461, 1.3605738878250122) | (-1.0052341222763062, 0.9942792057991028) |
| 203 | (-1.2799937725067139, 1.4421989917755127) | (-1.0009030103683472, 1.0006886720657349) |
| 204 | (-1.4959006309509277, 1.2481743097305298) | (-1.0037389993667603, 1.004176378250122) |
| 205 | (-1.336564302444458, 1.3556361198425293) | (-1.0106079578399658, 1.0082582235336304) |
| 206 | (-1.3791122436523438, 1.3538262844085693) | (-0.9972933530807495, 1.0063366889953613) |
| 207 | (-1.5169169902801514, 1.417389988899231) | (-0.9853416681289673, 1.0076725482940674) |
| 208 | (-1.3423489332199097, 1.3064743280410767) | (-0.9833009243011475, 1.034071683883667) |
| 209 | (-1.335217833518982, 1.294786810874939) | (-0.9997327327728271, 1.014747977256775) |
| 210 | (-1.3305208683013916, 1.4009631872177124) | (-0.9935685992240906, 1.008432149887085) |
| 211 | (-1.3112168312072754, 1.3375377655029297) | (-0.9980786442756653, 1.002718210220337) |
| 212 | (-1.3124204874038696, 1.3605451583862305) | (-1.0023845434188843, 1.007851243019104) |
| 213 | (-1.391365647315979, 1.2934813499450684) | (-1.0126254558563232, 0.9942508935928345) |
| 214 | (-1.2746710777282715, 1.3122966289520264) | (-1.000436782836914, 0.9988918900489807) |
| 215 | (-1.2815457582473755, 1.2878769636154175) | (-1.0052778720855713, 0.9857667684555054) |
| 216 | (-1.300210952758789, 1.3542346954345703) | (-0.9975203275680542, 1.0005016326904297) |
| 217 | (-1.3105013370513916, 1.4448150396347046) | (-1.0010772943496704, 0.994871199131012) |
| 218 | (-1.3713440895080566, 1.3145415782928467) | (-1.0034763813018799, 0.999509871006012) |
| 219 | (-1.2741773128509521, 1.3715217113494873) | (-0.993328869342804, 0.9952062964439392) |
| 220 | (-1.3532103300094604, 1.3045704364776611) | (-0.9616951942443848, 1.0245487689971924) |
| 221 | (-1.3040776252746582, 1.3181984424591064) | (-0.9053628444671631, 1.0548311471939087) |
| 222 | (-1.3959732055664062, 1.3126529455184937) | (-1.0099252541097022, 1.00391852871743) |
| 223 | (-1.4052133560180664, 1.3057639598846436) | (-1.0106050968170166, 1.0170515775680542) |
| 224 | (-1.4045178890228271, 1.3439565896987915) | (-1.0101516246795654, 1.0016772747039795) |
| 225 | (-1.285249948501587, 1.3140954971313477) | (-0.9929457306861877, 1.0161826610565186) |
| 226 | (-1.5067739486694336, 1.4287724494934082) | (-0.9424609541893005, 1.0399516820907593) |
| 227 | (-1.2958667278289795, 1.2416387796401978) | (-1.0108364820480347, 1.0049073696136475) |
| 228 | (-1.3508884906768799, 1.3625483512878418) | (-1.0053447484970093, 0.9961518049240112) |
| 229 | (-1.3664581775665283, 1.3643471002578735) | (-0.9997828602790833, 0.9943053722381592) |
| 230 | (-1.3538224697113037, 1.4813568592071533) | (-0.99784916639328, 1.0060646533966064) |
| 231 | (-1.4212148189544678, 1.3409184217453003) | (-1.0034834146499634, 1.0200133323669434) |
| 232 | (-1.367067575454712, 1.4189300537109375) | (-1.0019574165344238, 0.9871551990509033) |
| 233 | (-1.410024881362915, 1.3674125671386719) | (-0.9818676114082336, 0.9986133575439453) |
| 234 | (-1.4845526218414307, 1.2716503143310547) | (-0.9876353144645691, 1.0011316537857056) |
| 235 | (-1.3066630363464355, 1.3769361972808838) | (-0.9902039170265198, 0.9836552739143372) |
| 236 | (-1.3391668796539307, 1.3203436136245728) | (-0.9938201308250427, 0.9985287189483643) |
| 237 | (-1.3586952686309814, 1.3707728385925293) | (-0.9971149563789368, 0.9927896857261658) |
| 238 | (-1.362623929977417, 1.303527593612671) | (-0.9870563745498657, 0.9984396696090698) |
| 239 | (-1.295072317123413, 1.3778290748596191) | (-1.0046132802963257, 1.009740948677063) |
| 240 | (-1.3846696615219116, 1.4260557889938354) | (-1.0146679878234863, 1.0081239938735962) |
| 241 | (-1.3462259769439697, 1.322777509689331) | (-0.9989389181137085, 0.9953305125236511) |
| 242 | (-1.3942883014678955, 1.412887454032898) | (-0.9917837977409363, 1.0098060369491577) |
| 243 | (-1.3608977794647217, 1.2991341352462769) | (-1.0198124647140503, 0.9945645332336426) |
| 244 | (-1.2773749828338623, 1.3578674793243408) | (-1.001479983329773, 1.0028839111328125) |
| 245 | (-1.3811829090118408, 1.3911309242248535) | (-1.0098578929901123, 0.9932448863983154) |
| 246 | (-1.3062126636505127, 1.2866140604019165) | (-0.9651966691017151, 1.018398642539978) |
| 247 | (-1.3259363174438477, 1.2993565797805786) | (-0.8738111257553101, 1.0681649446487427) |
| 248 | (-1.3957560062408447, 1.4213078022003174) | (-1.0098098516464233, 1.006905198097229) |
| 249 | (-1.2843509912490845, 1.3664963245391846) | (-0.9877163171768188, 1.0044176578521729) |
| 250 | (-1.2920405864715576, 1.3754117488861084) | (-0.993835985660553, 1.0005441904067993) |
| 251 | (-1.3313555717468262, 1.2869434356689453) | (-1.0049968957901, 0.9929764270782471) |
| 252 | (-1.3776979446411133, 1.4015778303146362) | (-0.9957042336463928, 0.9962427616119385) |
| 253 | (-1.3136612176895142, 1.3498976230621338) | (-1.0003137588500977, 1.0007888078689575) |
| 254 | (-1.3584471940994263, 1.3430149555206299) | (-0.986796498298645, 1.007581114768982) |
| 255 | (-1.4139541387557983, 1.3743641376495361) | (-0.9993447065353394, 0.9771125316619873) |
| 256 | (-1.297072410583496, 1.2937992811203003) | (-0.9967437982559204, 0.9980777502059937) |
| 257 | (-1.358834147453308, 1.3048663139343262) | (-1.0002328157424927, 0.9729214906692505) |
| 258 | (-1.4146716594696045, 1.4694677591323853) | (-1.0050058364868164, 1.0040260553359985) |
| 259 | (-1.3953375816345215, 1.340034008026123) | (-0.9525063037872314, 0.9554315805435181) |
| 260 | (-1.3779664039611816, 1.3546154499053955) | (-1.0031070709228516, 1.0039244890213013) |
| 261 | (-1.4444444179534912, 1.3879854679107666) | (-1.0021049976348877, 1.0066391229629517) |
| 262 | (-1.4574775695800781, 1.3466449975967407) | (-1.005535364151001, 1.0075894594192505) |
| 263 | (-1.3388268947601318, 1.3137980699539185) | (-0.9826118350028992, 0.9845237135887146) |
| 264 | (-1.3078604936599731, 1.3131401538848877) | (-1.0002830028533936, 0.9917935132980347) |
| 265 | (-1.3357765674591064, 1.305829405784607) | (-0.9903300404548645, 1.0028198957443237) |
| 266 | (-1.4107211828231812, 1.3277344703674316) | (-1.0120329856872559, 1.0112433433532715) |
| 267 | (-1.3142744302749634, 1.393580436706543) | (-0.9875614643096924, 0.9893711805343628) |
| 268 | (-1.454823613166809, 1.3693441152572632) | (-1.0006526708602905, 1.0020053386688232) |
| 269 | (-1.348271131515503, 1.3980741500854492) | (-0.9888931512832642, 0.9932339787483215) |
| 270 | (-1.3100579977035522, 1.4094902276992798) | (-1.0011378526687622, 0.9941698312759399) |
| 271 | (-1.3270679712295532, 1.341535210609436) | (-0.9894970655441284, 1.033801555633545) |
| 272 | (-1.335829496383667, 1.3809090852737427) | (-0.9526415467262268, 1.0347250699996948) |
| 273 | (-1.3492519855499268, 1.3393384218215942) | (-0.9932449460029602, 0.9938353300094604) |
| 274 | (-1.3706032037734985, 1.5214674472808838) | (-0.9991532564163208, 1.011556625366211) |
| 275 | (-1.400455355644226, 1.4447678327560425) | (-1.01112699508667, 1.0007203817367554) |
| 276 | (-1.2771012783050537, 1.3250620365142822) | (-0.9905132055282593, 1.0072236061096191) |
| 277 | (-1.4452934265136719, 1.350646734237671) | (-0.9926025867462158, 1.025136947631836) |
| 278 | (-1.4308910369873047, 1.2799937725067139) | (-1.0043082237243652, 0.9855740666389465) |
| 279 | (-1.3216297626495361, 1.3026719093322754) | (-1.0000600814819336, 1.007161021232605) |
| 280 | (-1.3141741752624512, 1.2893108129501343) | (-1.0008906126022339, 1.0055195093154907) |
| 281 | (-1.3110431432724, 1.3644773960113525) | (-1.0007842779159546, 1.0000300407409668) |
| 282 | (-1.3797898292541504, 1.3371599912643433) | (-0.9810485243797302, 0.9744721055030823) |
| 283 | (-1.2625927925109863, 1.2729541063308716) | (-1.0111147165298462, 1.0077576637268066) |
| 284 | (-1.398635983467102, 1.3618026971817017) | (-0.994468629360199, 0.995866060256958) |
| 285 | (-1.3715076446533203, 1.3771651983261108) | (-0.9662668108940125, 0.9945423007011414) |
| 286 | (-1.2600888013839722, 1.3354566097259521) | (-1.0064046382904053, 0.9976197481155396) |
| 287 | (-1.2563207149505615, 1.4690074920654297) | (-1.0099252541007022, 1.00391852871043) |
| 288 | (-1.3230654001235962, 1.291566014289856) | (-1.0073240995407104, 0.9984162449836731) |
| 289 | (-1.5174612998962402, 1.3007813692092896) | (-0.9904520511627197, 1.0067843198776245) |
| 290 | (-1.3190245628356934, 1.4183073043823242) | (-1.0012826919555664, 0.982486367225647) |
| 291 | (-1.3599731922149658, 1.351576805114746) | (-0.9470503926277161, 1.0018607378005981) |
| 292 | (-1.3925074338912964, 1.348140001296997) | (-1.0079829692840576, 0.9940866231918335) |
| 293 | (-1.4595327377319336, 1.387532114982605) | (-1.0038172006607056, 1.0011882781982422) |
| 294 | (-1.3190809488296509, 1.2648119926452637) | (-0.9608869552612305, 0.9961000680923462) |
| 295 | (-1.3409876823425293, 1.3347630500793457) | (-1.0017110109329224, 1.0218576192855835) |
| 296 | (-1.3566513061523438, 1.3400092124938965) | (-1.0036863088607788, 1.0146286487579346) |
| 297 | (-1.3500059843063354, 1.2799227237701416) | (-0.9837813973426819, 0.9761678576469421) |
| 298 | (-1.3685295581817627, 1.2715402841567993) | (-0.9955076575279236, 1.0112662315368652) |
| 299 | (-1.393052577972412, 1.324843168258667) | (-0.9928490519523621, 1.0104602575302124) |
| 300 | (-1.3868669271469116, 1.3207393884658813) | (-1.0100141763687134, 1.005359172821045) |
| 301 | (-1.2967125177383423, 1.3803013563156128) | (-0.9045236110687256, 1.0399343967437744) |
| 302 | (-1.398498773574829, 1.3819812536239624) | (-0.9901494383811951, 1.0117822885513306) |
| 303 | (-1.4052743911743164, 1.5258615016937256) | (-1.012328863143921, 0.9952012896537781) |
| 304 | (-1.351229190826416, 1.2613283395767212) | (-0.9968162178993225, 1.0002732276916504) |
| 305 | (-1.3565075397491455, 1.3474819660186768) | (-0.9790088534355164, 0.9781907796859741) |
| 306 | (-1.3598850965499878, 1.3324229717254639) | (-1.0044101476669312, 0.9993529319763184) |
| 307 | (-1.3084665536880493, 1.3100988864898682) | (-0.9873334765434265, 0.9842069745063782) |
| 308 | (-1.2984600067138672, 1.3529605865478516) | (-1.002624273300171, 0.9910473227500916) |
| 309 | (-1.4202682971954346, 1.2737247943878174) | (-0.9941592812538147, 0.9948590397834778) |
| 310 | (-1.3059853315353394, 1.336251974105835) | (-1.009943962097168, 0.9992175102233887) |
| 311 | (-1.4534049034118652, 1.2988600730895996) | (-1.0043812990188599, 1.007790446281433) |
| 312 | (-1.39583158493042, 1.3001844882965088) | (-0.9997707605361938, 1.0001047849655151) |
| 313 | (-1.378697156906128, 1.2889821529388428) | (-1.0030189752578735, 0.9985317587852478) |
| 314 | (-1.305471658706665, 1.2563676834106445) | (-0.8838942646980286, 1.0605734586715698) |
| 315 | (-1.2646735906600952, 1.3561196327209473) | (-1.014977216720581, 0.9982416033744812) |
| 316 | (-1.292759656906128, 1.3144676685333252) | (-1.0087745189666748, 1.0087521076202393) |
| 317 | (-1.3325968980789185, 1.3849459886550903) | (-0.9961501359939575, 0.9949232935905457) |
| 318 | (-1.2992498874664307, 1.3626933097839355) | (-0.92631596326828, 1.0623397827148438) |
| 319 | (-1.3689424991607666, 1.31613028049469) | (-1.0141931772232056, 0.9897453188896179) |
| 320 | (-1.3435112237930298, 1.4082461595535278) | (-0.9981500506401062, 0.9955323338508606) |
| 321 | (-1.3675013780593872, 1.3008390665054321) | (-1.0024118423461914, 1.0054477453231812) |
| 322 | (-1.302789330482483, 1.3022445440292358) | (-1.0056877136230469, 1.0335147380828857) |
| 323 | (-1.2834935188293457, 1.4912140369415283) | (-1.0048528909683228, 0.9969948530197144) |
| 324 | (-1.311783790588379, 1.4214401245117188) | (-0.9954361915588379, 0.9878216981887817) |
| 325 | (-1.3359549045562744, 1.3358404636383057) | (-1.0038022994995117, 1.0135301351547241) |
| 326 | (-1.3664624691009521, 1.336284875869751) | (-0.9971156716346741, 0.9974750280380249) |
| 327 | (-1.2660691738128662, 1.3492627143859863) | (-0.9885342717170715, 0.9961889386177063) |
| 328 | (-1.4573413133621216, 1.308178186416626) | (-0.9947439432144165, 0.9996960759162903) |
| 329 | (-1.294952392578125, 1.2603132724761963) | (-1.0006455183029175, 1.0003292560577393) |
| 330 | (-1.3031086921691895, 1.368085503578186) | (-0.8647444844245911, 1.019270420074463) |
| 331 | (-1.3533625602722168, 1.3710341453552246) | (-0.9905939698219299, 1.0071510076522827) |
| 332 | (-1.3300755023956299, 1.3443070650100708) | (-0.9849857091903687, 0.995916485786438) |
| 333 | (-1.3365696668624878, 1.3736008405685425) | (-1.0132924318313599, 0.9983364343643188) |
| 334 | (-1.365966558456421, 1.3764877319335938) | (-1.0045428276062012, 0.9928879141807556) |
| 335 | (-1.353811502456665, 1.3567569255828857) | (-1.0240802764892578, 1.0172977447509766) |
| 336 | (-1.4040647745132446, 1.370356559753418) | (-0.9897081851959229, 1.0102741718292236) |
| 337 | (-1.5148417949676514, 1.3249106407165527) | (-0.9862602949142456, 0.988189697265625) |
| 338 | (-1.359166145324707, 1.4615702629089355) | (-0.986624002456665, 0.9869727492332458) |
| 339 | (-1.3374918699264526, 1.3134568929672241) | (-0.9903339147567749, 1.004596471786499) |
| 340 | (-1.3360092639923096, 1.3207859992980957) | (-0.9994843006134033, 1.0056861639022827) |
| 341 | (-1.483510971069336, 1.3204808235168457) | (-1.0015884637832642, 0.99947190284729) |
| 342 | (-1.3545420169830322, 1.3336458206176758) | (-0.9913472533226013, 0.9986611008644104) |
| 343 | (-1.3788197040557861, 1.2792551517486572) | (-0.9769814610481262, 1.0052956342697144) |
| 344 | (-1.4041945934295654, 1.2674953937530518) | (-1.0159729719161987, 1.0020595788955688) |
| 345 | (-1.3999491930007935, 1.3133666515350342) | (-1.0098252541007022, 1.00381852871043) |
| 346 | (-1.324379324913025, 1.4580700397491455) | (-1.0026434659957886, 1.0090621709823608) |
| 347 | (-1.3639869689941406, 1.3224987983703613) | (-0.9966335296630859, 0.9884900450706482) |
| 348 | (-1.29929780960083, 1.3388253450393677) | (-1.0006657838821411, 1.0137555599212646) |
| 349 | (-1.2510915994644165, 1.380838394165039) | (-1.0076205730438232, 1.0027977228164673) |
| 350 | (-1.3531687259674072, 1.349729299545288) | (-0.9868287444114685, 0.9944071173667908) |
| 351 | (-1.3651255369186401, 1.3383623361587524) | (-1.009272813796997, 1.000351071357727) |
| 352 | (-1.3720674514770508, 1.4521996974945068) | (-0.9956598877906799, 0.997035801410675) |
| 353 | (-1.308140516281128, 1.2787492275238037) | (-1.0195940732955933, 1.006447434425354) |
| 354 | (-1.2367993593215942, 1.4482356309890747) | (-0.9859819412231445, 0.9942251443862915) |
| 355 | (-1.3676663637161255, 1.3192706108093262) | (-0.9935685992240906, 0.9954193234443665) |
| 356 | (-1.3520944118499756, 1.3030576705932617) | (-0.9134398102760315, 1.0244200229644775) |
| 357 | (-1.3650102615356445, 1.3167532682418823) | (-0.9720332026481628, 1.0323110818862915) |
| 358 | (-1.2971220016479492, 1.368567705154419) | (-1.0069093704223633, 1.0136055946350098) |
| 359 | (-1.3583201169967651, 1.2443170547485352) | (-0.9594002366065979, 1.0030040740966797) |
| 360 | (-1.4538726806640625, 1.2464524507522583) | (-1.003348469734192, 1.0012632608413696) |
| 361 | (-1.2548117637634277, 1.396323561668396) | (-0.9747832417488098, 0.9947636127471924) |
| 362 | (-1.3142163753509521, 1.271424412727356) | (-0.9889355897903442, 0.995407223701477) |
| 363 | (-1.3253209590911865, 1.292360544204712) | (-0.9953580498695374, 0.9936255216598511) |
| 364 | (-1.351785659790039, 1.392494559288025) | (-0.9992929100990295, 1.0047932863235474) |
| 365 | (-1.3098723888397217, 1.308182954788208) | (-1.0111558437347412, 1.002013087272644) |
| 366 | (-1.357144832611084, 1.3252772092819214) | (-0.9925793409347534, 0.9929065704345703) |
| 367 | (-1.3004029989242554, 1.2414792776107788) | (-1.0160386562347412, 1.0063328742980957) |
| 368 | (-1.3099193572998047, 1.4420015811920166) | (-1.0005004405975342, 1.0079963207244873) |
| 369 | (-1.3818506002426147, 1.459014892578125) | (-1.0056772232055664, 1.0048797130584717) |
| 370 | (-1.4156213998794556, 1.4202467203140259) | (-1.0085400342941284, 1.0069597959518433) |
| 371 | (-1.2671889066696167, 1.5992590188980103) | (-0.9934474229812622, 0.9965150356292725) |
| 372 | (-1.4432568550109863, 1.464115858078003) | (-1.009934902191162, 1.0080286264419556) |
| 373 | (-1.4137063026428223, 1.3441376686096191) | (-0.9889823198318481, 0.9988541603088379) |
| 374 | (-1.3389445543289185, 1.283850908279419) | (-0.9958523511886597, 1.0004664659500122) |
| 375 | (-1.3814506530761719, 1.2783441543579102) | (-0.9974781274795532, 0.9839337468147278) |
| 376 | (-1.3177709579467773, 1.3513290882110596) | (-1.0037736892700195, 1.0131103992462158) |
| 377 | (-1.4633920192718506, 1.5573511123657227) | (-1.0044798851013184, 1.0045973062515259) |
| 378 | (-1.3768105506896973, 1.2919996976852417) | (-1.007030963897705, 1.0021594762802124) |
| 379 | (-1.3517017364501953, 1.3571971654891968) | (-1.0094748735427856, 1.0114794969558716) |
| 380 | (-1.3200651407241821, 1.3284505605697632) | (-1.0096805095672607, 1.0052155256271362) |
| 381 | (-1.3920984268188477, 1.292863130569458) | (-0.9783338308334351, 0.9881469011306763) |
| 382 | (-1.4853734970092773, 1.339777946472168) | (-0.9955866932868958, 0.9965572953224182) |
| 383 | (-1.4662386178970337, 1.2740310430526733) | (-1.000580072402954, 0.9882162809371948) |
| 384 | (-1.3054357767105103, 1.2730313539505005) | (-0.9956347942352295, 0.9985819458961487) |
| 385 | (-1.330897331237793, 1.4137104749679565) | (-0.9998303651809692, 1.0041922330856323) |
| 386 | (-1.3681923151016235, 1.2749583721160889) | (-0.9192116856575012, 1.0119234323501587) |
| 387 | (-1.3170278072357178, 1.384279489517212) | (-0.968021035194397, 1.0491275787353516) |
| 388 | (-1.3176615238189697, 1.3338675498962402) | (-1.0087448358535767, 1.0092157125473022) |
| 389 | (-1.3628392219543457, 1.2366523742675781) | (-0.9978338479995728, 0.9818204641342163) |
| 390 | (-1.418601155281067, 1.3436335325241089) | (-1.007643222808838, 0.9968917965888977) |
| 391 | (-1.4590586423873901, 1.305884838104248) | (-1.015960693359375, 1.0100959539413452) |
| 392 | (-1.2995851039886475, 1.2553852796554565) | (-1.002023696899414, 0.9874902367591858) |
| 393 | (-1.3465397357940674, 1.346443772315979) | (-1.0106582641601562, 1.0014088153839111) |
| 394 | (-1.325585126876831, 1.3212815523147583) | (-0.9962997436523438, 1.0020009279251099) |
| 395 | (-1.3813520669937134, 1.442586064338684) | (-0.9945296049118042, 0.9993234872817993) |
| 396 | (-1.44181489944458, 1.424674153327942) | (-1.001505970954895, 1.0001164674758911) |
| 397 | (-1.259443759918213, 1.3901710510253906) | (-1.0097252541007022, 1.00381752871043) |
| 398 | (-1.2468831539154053, 1.4011304378509521) | (-0.982610821723938, 0.9885244965553284) |
| 399 | (-1.358064889907837, 1.3610774278640747) | (-1.0129116773605347, 1.0066096782684326) |
| 400 | (-1.3775455951690674, 1.350114107131958) | (-0.9977566599845886, 1.0067658424377441) |
| 401 | (-1.2637040615081787, 1.2713687419891357) | (-0.9952306747436523, 0.9924299716949463) |
| 402 | (-1.2694038152694702, 1.3719757795333862) | (-0.9931509494781494, 1.0115047693252563) |
| 403 | (-1.2922683954238892, 1.301195740699768) | (-1.0141268968582153, 1.0084972381591797) |
| 404 | (-1.3329551219940186, 1.325010895729065) | (-0.9956920146942139, 1.0035161972045898) |
| 405 | (-1.3893779516220093, 1.424395203590393) | (-1.008244514465332, 1.0014197826385498) |
| 406 | (-1.2993003129959106, 1.3812907934188843) | (-0.9985606074333191, 0.991875171661377) |
| 407 | (-1.2845821380615234, 1.3080826997756958) | (-0.9325388073921204, 0.9732767343521118) |
| 408 | (-1.3104835748672485, 1.3897591829299927) | (-1.007017970085144, 1.0076462030410767) |
| 409 | (-1.3918620347976685, 1.392505168914795) | (-0.998532235622406, 0.9926853775978088) |
| 410 | (-1.2587389945983887, 1.442683219909668) | (-0.9892842769622803, 1.0052523612976074) |
| 411 | (-1.373530387878418, 1.3005008697509766) | (-0.9754279255867004, 0.9828107953071594) |
| 412 | (-1.3450148105621338, 1.383672833442688) | (-0.9913463592529297, 1.0003386735916138) |
| 413 | (-1.3223587274551392, 1.283452033996582) | (-1.0105886459350586, 0.9921102523803711) |
| 414 | (-1.4097529649734497, 1.352959156036377) | (-1.0132126808166504, 0.9931434988975525) |
| 415 | (-1.2935322523117065, 1.3957490921020508) | (-0.9989121556282043, 0.9917784929275513) |
| 416 | (-1.373714804649353, 1.4494813680648804) | (-0.9978604316711426, 0.9950095415115356) |
| 417 | (-1.316496729850769, 1.296608567237854) | (-0.9922881126403809, 1.006347894668579) |
| 418 | (-1.2662198543548584, 1.4066381454467773) | (-1.0015865564346313, 0.9990907907485962) |
| 419 | (-1.3069610595703125, 1.38151216506958) | (-0.9812785983085632, 1.016268253326416) |
| 420 | (-1.3481659889221191, 1.2952313423156738) | (-1.008949637413025, 0.9891147017478943) |
| 421 | (-1.3168094158172607, 1.3902137279510498) | (-1.0056848526000977, 1.0138952732086182) |
| 422 | (-1.320149302482605, 1.3230587244033813) | (-1.0132441520690918, 1.0017926692962646) |
| 423 | (-1.3446791172027588, 1.245070457458496) | (-0.9924747943878174, 0.9825713038444519) |
| 424 | (-1.274202823638916, 1.331012487411499) | (-1.009541630744934, 1.008280873298645) |
| 425 | (-1.3910832405090332, 1.2906616926193237) | (-1.0133650302886963, 0.9819967150688171) |
| 426 | (-1.437821388244629, 1.2823961973190308) | (-0.9915196895599365, 1.000817060470581) |
| 427 | (-1.3721060752868652, 1.274362325668335) | (-1.0113718509674072, 0.9960422515869141) |
| 428 | (-1.3113068342208862, 1.3507381677627563) | (-0.9881988763809204, 0.9972158670425415) |
| 429 | (-1.2676705121994019, 1.4833335876464844) | (-0.9994801878929138, 0.9971815347671509) |
| 430 | (-1.3187682628631592, 1.3511133193969727) | (-1.0019032955169678, 1.004265308380127) |
| 431 | (-1.315665602684021, 1.3173534870147705) | (-0.9957171082496643, 1.0136507749557495) |
| 432 | (-1.3562140464782715, 1.268869161605835) | (-0.990108847618103, 0.9996883273124695) |
| 433 | (-1.3142982721328735, 1.3443089723587036) | (-0.9929295778274536, 0.9937797784805298) |
| 434 | (-1.3151776790618896, 1.2992453575134277) | (-1.001409888267517, 1.0114413499832153) |
| 435 | (-1.3254579305648804, 1.2733006477355957) | (-0.9841220378875732, 0.9937703609466553) |
| 436 | (-1.297641634941101, 1.4236841201782227) | (-1.0031366348266602, 1.012462854385376) |
| 437 | (-1.2539641857147217, 1.364953875541687) | (-0.992500364780426, 0.9950236082077026) |
| 438 | (-1.3141499757766724, 1.3595681190490723) | (-0.9942182302474976, 1.0052857398986816) |
| 439 | (-1.308790683746338, 1.3390092849731445) | (-0.9914312958717346, 1.001430630683899) |
| 440 | (-1.3913235664367676, 1.4265823364257812) | (-1.006511926651001, 1.0042359828948975) |
| 441 | (-1.4026412963867188, 1.276036262512207) | (-1.0040870904922485, 0.9812222719192505) |
| 442 | (-1.4071643352508545, 1.2531434297561646) | (-0.9944006204605103, 0.9781733751296997) |
| 443 | (-1.311944603919983, 1.43966805934906) | (-1.0125782489776611, 0.9779647588729858) |
| 444 | (-1.3259721994400024, 1.2834031581878662) | (-1.0084947347640991, 0.995647132396698) |
| 445 | (-1.2756164073944092, 1.349062204360962) | (-1.001375675201416, 1.001753330230713) |
| 446 | (-1.4123116731643677, 1.3407469987869263) | (-0.9938183426856995, 0.9946820139884949) |
| 447 | (-1.346736192703247, 1.3084971904754639) | (-1.0019656419754028, 0.9939783215522766) |
| 448 | (-1.2581787109375, 1.3040781021118164) | (-0.9929172992706299, 0.9865790009498596) |
| 449 | (-1.396777868270874, 1.3874536752700806) | (-1.0101768970489502, 1.003161907196045) |
| 450 | (-1.3041093349456787, 1.395685076713562) | (-0.9901014566421509, 1.0098930597305298) |
| 451 | (-1.2740063667297363, 1.277787446975708) | (-0.9916970133781433, 1.0049803256988525) |
| 452 | (-1.28021240234375, 1.3763760328292847) | (-0.9987328052520752, 0.9962050914764404) |
| 453 | (-1.3169946670532227, 1.3259915113449097) | (-0.9956760406494141, 1.0011979341506958) |
| 454 | (-1.3474233150482178, 1.3181027173995972) | (-0.9975910782814026, 0.9959520697593689) |
| 455 | (-1.334735631942749, 1.2729090452194214) | (-0.896949291229248, 1.069802165031433) |
| 456 | (-1.3700429201126099, 1.371237874031067) | (-0.9973684549331665, 1.0046950578689575) |
| 457 | (-1.384652853012085, 1.3928816318511963) | (-1.0084078311920166, 1.0008423328399658) |
| 458 | (-1.3929740190505981, 1.3995332717895508) | (-1.0065439939498901, 1.0154577493667603) |
| 459 | (-1.3565202951431274, 1.3845875263214111) | (-1.0129282474517822, 0.9884798526763916) |
| 460 | (-1.3252471685409546, 1.2813483476638794) | (-0.9493444561958313, 1.042067289352417) |
| 461 | (-1.3413426876068115, 1.3567851781845093) | (-0.9905887842178345, 0.996483325958252) |
| 462 | (-1.3272989988327026, 1.477421522140503) | (-1.0046037435531616, 0.9931980967521667) |
| 463 | (-1.3275146484375, 1.2591190338134766) | (-1.0109769105911255, 0.9606324434280396) |
| 464 | (-1.3478591442108154, 1.4077632427215576) | (-1.0107988119125366, 0.9969642162322998) |
| 465 | (-1.3676382303237915, 1.303789734840393) | (-0.9811112880706787, 0.9888394474983215) |
| 466 | (-1.4715678691864014, 1.3202499151229858) | (-1.0012152194976807, 1.0070347785949707) |
| 467 | (-1.3167848587036133, 1.2665351629257202) | (-1.0077171325683594, 1.0046772956848145) |
| 468 | (-1.3324732780456543, 1.2797099351882935) | (-1.0032483339309692, 0.9944742918014526) |
| 469 | (-1.439469575881958, 1.3663487434387207) | (-0.9894285798072815, 1.0062893629074097) |
| 470 | (-1.3004872798919678, 1.2862586975097656) | (-0.9772205948829651, 0.9848541617393494) |
| 471 | (-1.2858976125717163, 1.3894065618515015) | (-0.9952435493469238, 0.9991611242294312) |
| 472 | (-1.3346883058547974, 1.3219316005706787) | (-1.0154085159301758, 1.0140948295593262) |
| 473 | (-1.4021726846694946, 1.5567069053649902) | (-0.9899213910102844, 0.9985706806182861) |
| 474 | (-1.4618438482284546, 1.511622428894043) | (-0.9893714189529419, 1.0023748874664307) |
| 475 | (-1.4096486568450928, 1.361286997795105) | (-1.0010011196136475, 0.9975190162658691) |
| 476 | (-1.5170671939849854, 1.3401777744293213) | (-0.9248129725456238, 1.058782696723938) |
| 477 | (-1.3032941818237305, 1.3551909923553467) | (-1.0065995454788208, 1.0104877948760986) |
| 478 | (-1.3777590990066528, 1.2901233434677124) | (-0.9885849356651306, 0.9954615235328674) |
| 479 | (-1.3269994258880615, 1.3451005220413208) | (-0.9586339592933655, 0.9968698620796204) |
| 480 | (-1.372331976890564, 1.3725333213806152) | (-1.0237548351287842, 1.0061837434768677) |
| 481 | (-1.3655787706375122, 1.443929672241211) | (-1.0001152753829956, 1.003326416015625) |
| 482 | (-1.3003764152526855, 1.377227783203125) | (-1.0097804069519043, 0.9981294870376587) |
| 483 | (-1.3599929809570312, 1.424458384513855) | (-1.0022711753845215, 0.9921948313713074) |
| 484 | (-1.4916105270385742, 1.3165795803070068) | (-0.9882009625434875, 0.9859839081764221) |
| 485 | (-1.3377549648284912, 1.2533003091812134) | (-0.9945465922355652, 1.0121243000030518) |
| 486 | (-1.3651350736618042, 1.3226356506347656) | (-1.0069143772125244, 0.9874226450920105) |
| 487 | (-1.3942899703979492, 1.411935806274414) | (-1.0006868839263916, 0.996035635471344) |
| 488 | (-1.465048909187317, 1.418819546699524) | (-1.0007606744766235, 0.9896190166473389) |
| 489 | (-1.3448114395141602, 1.4637162685394287) | (-1.0074317455291748, 1.0002267360687256) |
| 490 | (-1.3289332389831543, 1.2846882343292236) | (-1.0026659965515137, 1.0210856199264526) |
| 491 | (-1.373506784439087, 1.387058973312378) | (-1.0097252641007022, 1.00381652871043) |
| 492 | (-1.4267938137054443, 1.4177167415618896) | (-0.9875906705856323, 0.9947702884674072) |
| 493 | (-1.3176984786987305, 1.4529643058776855) | (-0.9915918111801147, 0.9971482753753662) |
| 494 | (-1.26900053024292, 1.3523653745651245) | (-1.0143704414367676, 1.011488437652588) |
| 495 | (-1.3054442405700684, 1.3151326179504395) | (-0.9777218103408813, 0.9767135381698608) |
| 496 | (-1.3530454635620117, 1.4546884298324585) | (-0.9950898885726929, 0.9883689284324646) |
| 497 | (-1.278407335281372, 1.4449701309204102) | (-0.9817085862159729, 0.9192749857902527) |
| 498 | (-1.3655529022216797, 1.3086752891540527) | (-1.001844048500061, 0.9916096329689026) |
| 499 | (-1.2978917360305786, 1.3443915843963623) | (-1.0058064460754395, 1.0124694108963013) |
| 500 | (-1.34653902053833, 1.3598421812057495) | (-0.9984822869300842, 0.9984436631202698) |
| 501 | (-1.372138500213623, 1.442349910736084) | (-0.9959689378738403, 0.990826427936554) |
| 502 | (-1.3351343870162964, 1.3476024866104126) | (-0.9585639834403992, 0.9900230169296265) |
| 503 | (-1.3993667364120483, 1.3274673223495483) | (-1.003829836845398, 0.9847605228424072) |
| 504 | (-1.2283474206924438, 1.4144750833511353) | (-0.9517577290534973, 1.0233572721481323) |
| 505 | (-1.2997276782989502, 1.415611982345581) | (-0.9932822585105896, 1.0107536315917969) |
| 506 | (-1.3253679275512695, 1.3576549291610718) | (-1.006799340248108, 1.007951259613037) |
| 507 | (-1.3192545175552368, 1.3700947761535645) | (-0.9391422867774963, 1.0091685056686401) |
| 508 | (-1.349363088607788, 1.3658511638641357) | (-0.9870976805686951, 0.9812521934509277) |
| 509 | (-1.3878544569015503, 1.4808589220046997) | (-0.9987072348594666, 1.0063010454177856) |
| 510 | (-1.368922233581543, 1.3728299140930176) | (-0.9971333742141724, 0.9982007741928101) |
| 511 | (-1.4493155479431152, 1.3011455535888672) | (-1.028848648071289, 1.041750431060791) |
| 512 | (-1.3346762657165527, 1.431370496749878) | (-1.0057129859924316, 1.0077115297317505) |
| 513 | (-1.3680086135864258, 1.3436102867126465) | (-1.0028085708618164, 0.977527916431427) |
| 514 | (-1.3998777866363525, 1.4188082218170166) | (-1.0146937370300293, 0.9689990282058716) |
| 515 | (-1.3676649332046509, 1.2997183799743652) | (-1.0109165906906128, 1.012628197669983) |
| 516 | (-1.4058607816696167, 1.2903162240982056) | (-0.9937554597854614, 0.9979706406593323) |
| 517 | (-1.3161612749099731, 1.461979866027832) | (-0.9966936707496643, 1.0070626735687256) |
| 518 | (-1.2834815979003906, 1.344871163368225) | (-0.9855530858039856, 0.9962395429611206) |
| 519 | (-1.3077256679534912, 1.3322205543518066) | (-1.0020986795425415, 0.9915695190429688) |
| 520 | (-1.3545466661453247, 1.3728173971176147) | (-0.9722863435745239, 1.0061835050582886) |
| 521 | (-1.448005199432373, 1.3131115436553955) | (-0.9909120202064514, 0.9962444305419922) |
| 522 | (-1.343311071395874, 1.450641393661499) | (-1.0103813409805298, 1.0112711191177368) |
| 523 | (-1.475342035293579, 1.3467073440551758) | (-1.0008968114852905, 0.9583845734596252) |
| 524 | (-1.3126685619354248, 1.2919342517852783) | (-0.9987534284591675, 1.0055962800979614) |
| 525 | (-1.2919000387191772, 1.425358772277832) | (-1.0026075839996338, 1.0114165544509888) |
| 526 | (-1.3465924263000488, 1.3216524124145508) | (-1.000736117362976, 0.9943543672561646) |
| 527 | (-1.3764126300811768, 1.260657787322998) | (-0.9998412132263184, 0.9891675710678101) |
| 528 | (-1.3093992471694946, 1.4026429653167725) | (-0.9983487129211426, 1.0092427730560303) |
| 529 | (-1.322925090789795, 1.379165768623352) | (-1.005703330039978, 1.0086232423782349) |
| 530 | (-1.3973561525344849, 1.3200827836990356) | (-0.9853148460388184, 1.0207184553146362) |
| 531 | (-1.503401756286621, 1.4176690578460693) | (-1.0136163234710693, 1.006500482559204) |
| 532 | (-1.4443011283874512, 1.2684086561203003) | (-0.9901152849197388, 0.9979828000068665) |
| 533 | (-1.3331609964370728, 1.3289175033569336) | (-1.0039516687393188, 1.0114772319793701) |
| 534 | (-1.3216736316680908, 1.3399550914764404) | (-0.9954173564910889, 0.9841688275337219) |
| 535 | (-1.3478035926818848, 1.3291683197021484) | (-1.0097252541007022, 1.00381652571043) |
| 536 | (-1.3497616052627563, 1.232011079788208) | (-1.0232369899749756, 0.993545651435852) |
| 537 | (-1.3603899478912354, 1.397791862487793) | (-1.0126127004623413, 1.0041306018829346) |
| 538 | (-1.3507106304168701, 1.4037864208221436) | (-1.0106502771377563, 1.007948637008667) |
| 539 | (-1.2943859100341797, 1.3404405117034912) | (-1.012630581855774, 0.9763116240501404) |
| 540 | (-1.2955102920532227, 1.4348666667938232) | (-0.9974470138549805, 0.9937030673027039) |
| 541 | (-1.32752525806427, 1.3081042766571045) | (-1.009415864944458, 1.0089468955993652) |
| 542 | (-1.3695526123046875, 1.3053089380264282) | (-0.9992002248764038, 1.007971167564392) |
| 543 | (-1.231951355934143, 1.3028894662857056) | (-1.0175737142562866, 1.0029717683792114) |
| 544 | (-1.3170785903930664, 1.3273764848709106) | (-1.0097252541004022, 1.00381652541043) |
| 545 | (-1.2706143856048584, 1.3162604570388794) | (-0.9956154227256775, 1.015972375869751) |
| 546 | (-1.3643985986709595, 1.2875542640686035) | (-1.0089383125305176, 1.0031377077102661) |
| 547 | (-1.3496463298797607, 1.4007004499435425) | (-1.0263433456420898, 0.9978838562965393) |
| 548 | (-1.3127743005752563, 1.4469108581542969) | (-1.0007716417312622, 1.0022282600402832) |
| 549 | (-1.2755934000015259, 1.4051120281219482) | (-0.9922101497650146, 0.9997491240501404) |
| 550 | (-1.2691243886947632, 1.3515163660049438) | (-1.0182377099990845, 1.0052160024642944) |
| 551 | (-1.4375452995300293, 1.3375102281570435) | (-0.9982053637504578, 0.9834566116333008) |
| 552 | (-1.3415451049804688, 1.3443669080734253) | (-1.006169319152832, 1.0033516883850098) |
| 553 | (-1.2835413217544556, 1.4282578229904175) | (-1.0097251541004022, 1.00381612541043) |
| 554 | (-1.3790491819381714, 1.3171619176864624) | (-1.0032707452774048, 0.9976755976676941) |
| 555 | (-1.2984098196029663, 1.2851475477218628) | (-1.0128127336502075, 0.9969049096107483) |
| 556 | (-1.3124622106552124, 1.3210790157318115) | (-0.9875052571296692, 0.9789100885391235) |
| 557 | (-1.3065181970596313, 1.2685036659240723) | (-0.9733678698539734, 0.9941835403442383) |
| 558 | (-1.352765679359436, 1.4035204648971558) | (-1.0016705989837646, 1.0018106698989868) |
| 559 | (-1.3087021112442017, 1.3363232612609863) | (-1.0037107467651367, 1.0015918016433716) |
| 560 | (-1.3232311010360718, 1.3562291860580444) | (-1.0151269435882568, 0.98793625831604) |
| 561 | (-1.4002647399902344, 1.3335143327713013) | (-0.9969744682312012, 1.0063252449035645) |
| 562 | (-1.3097689151763916, 1.311660885810852) | (-1.0080965757369995, 1.0028051137924194) |
| 563 | (-1.4622184038162231, 1.4581260681152344) | (-1.014074683189392, 0.9889492392539978) |
| 564 | (-1.3565363883972168, 1.4126205444335938) | (-0.8993086814880371, 1.0893807411193848) |
| 565 | (-1.2988455295562744, 1.341015100479126) | (-1.0276856422424316, 1.0186527967453003) |
| 566 | (-1.3121718168258667, 1.3628973960876465) | (-0.9999411106109619, 1.0095576047897339) |
| 567 | (-1.3051236867904663, 1.3472356796264648) | (-1.002800703048706, 1.0046515464782715) |
| 568 | (-1.3449469804763794, 1.3697047233581543) | (-1.0002915859222412, 0.9984106421470642) |
| 569 | (-1.2849940061569214, 1.2706812620162964) | (-1.0081489086151123, 0.9586203098297119) |
| 570 | (-1.3008394241333008, 1.4101030826568604) | (-1.0231086015701294, 1.0074355602264404) |
| 571 | (-1.3344361782073975, 1.3070037364959717) | (-0.9926998615264893, 0.9926596879959106) |
| 572 | (-1.3158518075942993, 1.2925693988800049) | (-0.977723240852356, 0.9817760586738586) |
| 573 | (-1.2988804578781128, 1.3441970348358154) | (-1.0041894912719727, 1.0035817623138428) |
| 574 | (-1.2864603996276855, 1.4256813526153564) | (-0.9958043098449707, 1.003190517425537) |
| 575 | (-1.33366060256958, 1.2805616855621338) | (-0.903683066368103, 1.0757267475128174) |
| 576 | (-1.3117903470993042, 1.3560508489608765) | (-0.9997615218162537, 0.9898729920387268) |
| 577 | (-1.3192050457000732, 1.2583019733428955) | (-1.0033667087554932, 1.0057275295257568) |
| 578 | (-1.3848206996917725, 1.4883439540863037) | (-0.9390734434127808, 0.9515997171401978) |
| 579 | (-1.3032763004302979, 1.3640557527542114) | (-1.0014698505401611, 0.9966728687286377) |
| 580 | (-1.3000640869140625, 1.3557935953140259) | (-1.006846308708191, 1.0046985149383545) |
| 581 | (-1.3657293319702148, 1.3573168516159058) | (-0.9889617562294006, 1.008108377456665) |
| 582 | (-1.4278761148452759, 1.3135508298873901) | (-1.026550054550171, 0.9939420223236084) |
| 583 | (-1.3522292375564575, 1.3809163570404053) | (-1.0077307224273682, 0.9990387558937073) |
| 584 | (-1.366951823234558, 1.3597707748413086) | (-0.9947319030761719, 1.0149582624435425) |
| 585 | (-1.3616664409637451, 1.3368377685546875) | (-0.9965683817863464, 0.9925663471221924) |
| 586 | (-1.4032284021377563, 1.2792397737503052) | (-0.9922943115234375, 1.0038751363754272) |
| 587 | (-1.2913566827774048, 1.2469482421875) | (-0.986823320388794, 0.9840205311775208) |
| 588 | (-1.3554242849349976, 1.3179092407226562) | (-0.9893595576286316, 0.9983147978782654) |
| 589 | (-1.3870664834976196, 1.3103771209716797) | (-0.9539996385574341, 0.9556427001953125) |
| 590 | (-1.3491573333740234, 1.327874779701233) | (-1.0002716779708862, 1.001723051071167) |
| 591 | (-1.3577229976654053, 1.265954613685608) | (-1.007245659828186, 0.9792888760566711) |
| 592 | (-1.330536961555481, 1.3569104671478271) | (-0.9851809144020081, 1.0267070531845093) |
| 593 | (-1.2922179698944092, 1.355992317199707) | (-0.9914060831069946, 0.9968929290771484) |
| 594 | (-1.291015625, 1.314664363861084) | (-1.0028632879257202, 1.00355863571167) |
| 595 | (-1.3164615631103516, 1.261547565460205) | (-0.988821268081665, 0.9881105422973633) |
| 596 | (-1.332619071006775, 1.3277506828308105) | (-1.001226544380188, 0.9986891746520996) |
| 597 | (-1.299299955368042, 1.3722693920135498) | (-0.9823426604270935, 1.0115957260131836) |
| 598 | (-1.2964599132537842, 1.322120189666748) | (-0.9910200834274292, 0.9934155941009521) |
| 599 | (-1.3240021467208862, 1.276242733001709) | (-0.9862937331199646, 0.9950703382492065) |
| 600 | (-1.3527623414993286, 1.2896050214767456) | (-0.9952784776687622, 0.9826239347457886) |
| 601 | (-1.3432804346084595, 1.328295350074768) | (-0.9722882509231567, 0.9801458716392517) |
| 602 | (-1.2867097854614258, 1.4102567434310913) | (-0.9907861948013306, 0.9861749410629272) |
| 603 | (-1.408597707748413, 1.385299563407898) | (-0.989011824131012, 1.0005431175231934) |
| 604 | (-1.3987663984298706, 1.4307315349578857) | (-0.9700429439544678, 1.0203325748443604) |
| 605 | (-1.3706576824188232, 1.3692290782928467) | (-0.8968163728713989, 1.0546541213989258) |
| 606 | (-1.3177664279937744, 1.4162721633911133) | (-0.9989712238311768, 1.006056785583496) |
| 607 | (-1.4675312042236328, 1.5746738910675049) | (-1.000335931777954, 1.0081571340560913) |
| 608 | (-1.3816291093826294, 1.3621642589569092) | (-1.0065202713012695, 1.0066142082214355) |
| 609 | (-1.3320910930633545, 1.3527015447616577) | (-0.9975067973136902, 1.0124794244766235) |
| 610 | (-1.3240407705307007, 1.3781487941741943) | (-0.9664209485054016, 1.0262000560760498) |
| 611 | (-1.300715684890747, 1.397969365119934) | (-0.8928126096725464, 1.0998769998550415) |
| 612 | (-1.3438947200775146, 1.3198119401931763) | (-1.0081291198730469, 0.9920579195022583) |
| 613 | (-1.32814621925354, 1.3219976425170898) | (-1.0075769424438477, 1.0006868839263916) |
| 614 | (-1.3438971042633057, 1.3054680824279785) | (-0.9978708028793335, 0.9933635592460632) |
| 615 | (-1.3591927289962769, 1.3683046102523804) | (-0.8538929224014282, 1.1462502479553223) |
| 616 | (-1.377371072769165, 1.402374505996704) | (-1.0071381330490112, 0.9922978281974792) |
| 617 | (-1.4540084600448608, 1.3157488107681274) | (-0.9870761632919312, 0.9874871969223022) |
| 618 | (-1.347885012626648, 1.3230761289596558) | (-1.003523826599121, 0.9939172267913818) |
| 619 | (-1.3958791494369507, 1.3527085781097412) | (-1.0005615949630737, 1.0050028562545776) |
| 620 | (-1.3705384731292725, 1.2967358827590942) | (-1.0083982944488525, 1.0042550563812256) |
| 621 | (-1.3308415412902832, 1.354548692703247) | (-1.021949291229248, 1.01221764087677) |
| 622 | (-1.3038350343704224, 1.3572970628738403) | (-1.0023657083511353, 0.9920693039894104) |
| 623 | (-1.2528893947601318, 1.2993755340576172) | (-0.9982720017433167, 0.995093584060669) |
| 624 | (-1.3909567594528198, 1.4158731698989868) | (-1.001673936843872, 1.005132794380188) |
| 625 | (-1.306426763534546, 1.276010513305664) | (-0.9968165159225464, 0.9954125285148621) |
| 626 | (-1.4873318672180176, 1.3919044733047485) | (-1.0072076320648193, 1.0054491758346558) |
| 627 | (-1.3598591089248657, 1.4775645732879639) | (-0.9948065876960754, 0.9980846047401428) |
| 628 | (-1.3190661668777466, 1.3754850625991821) | (-1.00696861743927, 1.0073970556259155) |
| 629 | (-1.4093499183654785, 1.3413314819335938) | (-1.0082279443740845, 0.9996891617774963) |
| 630 | (-1.2552863359451294, 1.3682211637496948) | (-0.9841133952140808, 1.0013002157211304) |
| 631 | (-1.306612253189087, 1.306515097618103) | (-0.9320261478424072, 1.048693060874939) |
| 632 | (-1.366450309753418, 1.3356881141662598) | (-1.0137743949890137, 0.9886268377304077) |
| 633 | (-1.2980207204818726, 1.4029467105865479) | (-1.0012983083724976, 1.0160571336746216) |
| 634 | (-1.309278964996338, 1.3319462537765503) | (-0.9876689910888672, 0.9939903020858765) |
| 635 | (-1.4120368957519531, 1.3237330913543701) | (-1.0220805406570435, 1.0026201009750366) |
| 636 | (-1.3347179889678955, 1.3791627883911133) | (-1.0045111179351807, 1.0029164552688599) |
| 637 | (-1.2612872123718262, 1.268399715423584) | (-0.9914590120315552, 0.998392641544342) |
| 638 | (-1.2349258661270142, 1.363247036933899) | (-0.996027946472168, 1.008296251296997) |
| 639 | (-1.3251060247421265, 1.3512132167816162) | (-0.9813288450241089, 0.9814608693122864) |
| 640 | (-1.2831538915634155, 1.2895424365997314) | (-1.0014663934707642, 0.9967399835586548) |
| 641 | (-1.287007451057434, 1.2413396835327148) | (-1.016027808189392, 1.0061101913452148) |
| 642 | (-1.3856992721557617, 1.3502204418182373) | (-0.9934214949607849, 0.9924700260162354) |
| 643 | (-1.3792126178741455, 1.3938708305358887) | (-1.0167349576950073, 1.0158673524856567) |
| 644 | (-1.2968859672546387, 1.4432125091552734) | (-0.9855844378471375, 0.993577241897583) |
| 645 | (-1.2630136013031006, 1.4588863849639893) | (-0.999780535697937, 0.9960081577301025) |
| 646 | (-1.3115968704223633, 1.260957956314087) | (-0.9783490300178528, 1.0063852071762085) |
| 647 | (-1.4099820852279663, 1.3528341054916382) | (-0.9952268600463867, 0.9771672487258911) |
| 648 | (-1.3181791305541992, 1.334731101989746) | (-1.0217561721801758, 1.0230591297149658) |
| 649 | (-1.3629720211029053, 1.2764278650283813) | (-0.9959574937820435, 1.0038007497787476) |
| 650 | (-1.317517876625061, 1.5291670560836792) | (-1.0027799606323242, 0.9939108490943909) |
| 651 | (-1.325546145439148, 1.3195785284042358) | (-0.9944034218788147, 1.003655195236206) |
| 652 | (-1.3272262811660767, 1.3887176513671875) | (-0.8644773364067078, 1.0576814413070679) |
| 653 | (-1.3388900756835938, 1.404118537902832) | (-0.9995815753936768, 0.9955055117607117) |
| 654 | (-1.3104335069656372, 1.2686389684677124) | (-1.0101439952850342, 1.000665545463562) |
| 655 | (-1.4446654319763184, 1.439617395401001) | (-1.0028053522109985, 1.0145410299301147) |
| 656 | (-1.3772683143615723, 1.426047682762146) | (-0.9942207336425781, 1.009791612625122) |
| 657 | (-1.366792917251587, 1.3944977521896362) | (-0.9986430406570435, 1.0018337965011597) |
| 658 | (-1.2815911769866943, 1.325810432434082) | (-1.0177116394042969, 0.9933842420578003) |
| 659 | (-1.3989861011505127, 1.4106037616729736) | (-1.0112676620483398, 1.0018799304962158) |
| 660 | (-1.2667744159698486, 1.2980014085769653) | (-1.004812479019165, 0.9966785311698914) |
| 661 | (-1.3822022676467896, 1.349551796913147) | (-0.9910879731178284, 1.0052621364593506) |
| 662 | (-1.497790813446045, 1.3229527473449707) | (-1.009216070175171, 1.0046478509902954) |
| 663 | (-1.3642433881759644, 1.4103444814682007) | (-0.9178450703620911, 1.0605919361114502) |
| 664 | (-1.3659571409225464, 1.3761780261993408) | (-1.0093467235565186, 0.9945486783981323) |
| 665 | (-1.4323073625564575, 1.4341747760772705) | (-1.0044077634811401, 1.0156269073486328) |
| 666 | (-1.3292608261108398, 1.3990567922592163) | (-1.0052560567855835, 1.0090017318725586) |
| 667 | (-1.321580410003662, 1.335845947265625) | (-1.0044454336166382, 0.9927417635917664) |
| 668 | (-1.324885368347168, 1.3433005809783936) | (-1.0097251541034022, 1.00381612341043) |
| 669 | (-1.262191891670227, 1.5385184288024902) | (-0.9932405948638916, 0.9883573651313782) |
| 670 | (-1.3032728433609009, 1.3885747194290161) | (-1.0024100542068481, 1.0096880197525024) |
| 671 | (-1.4790678024291992, 1.3413453102111816) | (-0.949707567691803, 1.063295602798462) |
| 672 | (-1.2439348697662354, 1.3908308744430542) | (-0.9932018518447876, 0.9899884462356567) |
| 673 | (-1.301387071609497, 1.2828657627105713) | (-0.99444580078125, 0.977197527885437) |
| 674 | (-1.3061695098876953, 1.2774548530578613) | (-1.0030183792114258, 0.992279589176178) |
| 675 | (-1.2808583974838257, 1.2757948637008667) | (-0.9955973029136658, 1.0090582370758057) |
| 676 | (-1.3549104928970337, 1.4384524822235107) | (-0.9962154626846313, 1.0122172832489014) |
| 677 | (-1.263871431350708, 1.2673828601837158) | (-0.9422976970672607, 1.0291308164596558) |
| 678 | (-1.271230697631836, 1.3977863788604736) | (-0.9874817132949829, 1.0043625831604004) |
| 679 | (-1.348038911819458, 1.3102471828460693) | (-0.9978747367858887, 1.003966212272644) |
| 680 | (-1.3041388988494873, 1.3914390802383423) | (-0.9793078899383545, 0.9981895685195923) |
| 681 | (-1.2795312404632568, 1.2403508424758911) | (-0.9855168461799622, 0.9801771640777588) |
| 682 | (-1.3082973957061768, 1.3720546960830688) | (-1.002798080444336, 0.9997454881668091) |
| 683 | (-1.3865175247192383, 1.3482226133346558) | (-0.9945721626281738, 1.0073347091674805) |
| 684 | (-1.3190351724624634, 1.3647279739379883) | (-0.9965730309486389, 0.9977595806121826) |
| 685 | (-1.3258904218673706, 1.3856532573699951) | (-1.0022544860839844, 0.9979653358459473) |
| 686 | (-1.407757043838501, 1.2501096725463867) | (-1.006284236907959, 0.9879716634750366) |
| 687 | (-1.3611081838607788, 1.265178918838501) | (-1.006818175315857, 0.9838684797286987) |
| 688 | (-1.2705605030059814, 1.3481237888336182) | (-1.0043361186981201, 1.0038719177246094) |
| 689 | (-1.2667359113693237, 1.356016993522644) | (-1.0063759088516235, 0.9364166259765625) |
| 690 | (-1.3274791240692139, 1.2419400215148926) | (-1.0019172430038452, 0.9995088577270508) |
| 691 | (-1.3850369453430176, 1.3721667528152466) | (-0.9971277117729187, 1.0121251344680786) |
| 692 | (-1.2804847955703735, 1.2932517528533936) | (-1.0020172595977783, 1.0014997720718384) |
| 693 | (-1.3647602796554565, 1.3012382984161377) | (-0.9732810258865356, 1.0294750928878784) |
| 694 | (-1.3175150156021118, 1.2817051410675049) | (-0.9759175777435303, 0.9791906476020813) |
| 695 | (-1.3799376487731934, 1.4418649673461914) | (-1.0097251541034222, 1.00381612241043) |
| 696 | (-1.3985309600830078, 1.253619909286499) | (-1.0043822526931763, 0.9929468035697937) |
| 697 | (-1.3409693241119385, 1.2777488231658936) | (-1.0207194089889526, 1.0006303787231445) |
| 698 | (-1.3824200630187988, 1.4434616565704346) | (-0.9406784176826477, 1.0092087984085083) |
| 699 | (-1.3148789405822754, 1.316942572593689) | (-1.003554105758667, 0.9877001047134399) |
| 700 | (-1.28190016746521, 1.272972583770752) | (-0.9852569699287415, 0.9881536364555359) |
| 701 | (-1.3353261947631836, 1.4873242378234863) | (-1.0087496042251587, 0.9983874559402466) |
| 702 | (-1.409083604812622, 1.265925407409668) | (-0.9950637221336365, 0.9983826279640198) |
| 703 | (-1.263841152191162, 1.3421844244003296) | (-1.00127112865448, 0.9918286204338074) |
| 704 | (-1.3702164888381958, 1.3962724208831787) | (-1.003235101699829, 1.025638222694397) |
| 705 | (-1.4203224182128906, 1.3554238080978394) | (-0.9719199538230896, 0.9753104448318481) |
| 706 | (-1.3065470457077026, 1.3838167190551758) | (-0.9999715685844421, 1.0136300325393677) |
| 707 | (-1.3444263935089111, 1.3106871843338013) | (-1.035563588142395, 0.9904783368110657) |
| 708 | (-1.3405457735061646, 1.3472464084625244) | (-1.002366542816162, 1.0001755952835083) |
| 709 | (-1.3340390920639038, 1.3811837434768677) | (-0.9304890036582947, 1.0139493942260742) |
| 710 | (-1.4619808197021484, 1.3498376607894897) | (-1.0026346445083618, 1.0032161474227905) |
| 711 | (-1.305658221244812, 1.3532239198684692) | (-0.9905501008033752, 1.0030007362365723) |
| 712 | (-1.3305684328079224, 1.4486771821975708) | (-0.9946197867393494, 0.9987402558326721) |
| 713 | (-1.3538625240325928, 1.29421067237854) | (-1.0133694410324097, 1.0041052103042603) |
| 714 | (-1.2778077125549316, 1.2872098684310913) | (-0.9645431041717529, 1.0081636905670166) |
| 715 | (-1.3678357601165771, 1.3728846311569214) | (-0.9918988347053528, 0.954571008682251) |
| 716 | (-1.3093454837799072, 1.3109569549560547) | (-1.0097251541034822, 1.00381618241043) |
| 717 | (-1.2738571166992188, 1.3281610012054443) | (-0.9929269552230835, 0.9820961952209473) |
| 718 | (-1.3474078178405762, 1.3332691192626953) | (-0.9934442639350891, 0.993205189704895) |
| 719 | (-1.4491685628890991, 1.3590034246444702) | (-1.0033376216888428, 1.0076189041137695) |
| 720 | (-1.3000249862670898, 1.3668547868728638) | (-0.9846899509429932, 1.002372145652771) |
| 721 | (-1.2834023237228394, 1.439998745918274) | (-1.014095664024353, 1.0040416717529297) |
| 722 | (-1.376774787902832, 1.403786063194275) | (-0.9721263647079468, 0.9885855913162231) |
| 723 | (-1.4149670600891113, 1.466163158416748) | (-0.9988282918930054, 1.0081195831298828) |
| 724 | (-1.2869043350219727, 1.3186904191970825) | (-0.9602049589157104, 1.0196764469146729) |
| 725 | (-1.493291974067688, 1.3326338529586792) | (-0.974614143371582, 0.9784048795700073) |
| 726 | (-1.2454118728637695, 1.257539987564087) | (-0.9897916316986084, 0.9945828318595886) |
| 727 | (-1.3187453746795654, 1.470720648765564) | (-0.9992423057556152, 0.9983974099159241) |
| 728 | (-1.2768070697784424, 1.3301551342010498) | (-1.0035656690597534, 0.9974125027656555) |
| 729 | (-1.475651741027832, 1.2761306762695312) | (-0.9827833771705627, 1.0253454446792603) |
| 730 | (-1.4628956317901611, 1.3518848419189453) | (-0.9924221038818359, 1.0041333436965942) |
| 731 | (-1.3906989097595215, 1.3636308908462524) | (-1.0073398351669312, 1.0065096616744995) |
| 732 | (-1.377476453781128, 1.3113363981246948) | (-1.0089224576950073, 1.01234769821167) |
| 733 | (-1.4302167892456055, 1.3829697370529175) | (-0.9913609027862549, 0.9842227101325989) |
| 734 | (-1.3525103330612183, 1.3565174341201782) | (-1.0013822317123413, 1.0045779943466187) |
| 735 | (-1.3266477584838867, 1.4389376640319824) | (-0.9972411394119263, 1.0101723670959473) |
| 736 | (-1.3570533990859985, 1.3740501403808594) | (-0.9978036284446716, 1.0187406539916992) |
| 737 | (-1.3584985733032227, 1.3010104894638062) | (-0.9917532801628113, 1.0011208057403564) |
| 738 | (-1.3677659034729004, 1.3536841869354248) | (-1.0014359951019287, 0.9989301562309265) |
| 739 | (-1.3717166185379028, 1.3512558937072754) | (-0.9947190880775452, 1.0003582239151) |
| 740 | (-1.316818118095398, 1.2641804218292236) | (-1.0097291541034822, 1.00381618941043) |
| 741 | (-1.3646843433380127, 1.3083512783050537) | (-0.9850363731384277, 0.9811128377914429) |
| 742 | (-1.270979642868042, 1.3361179828643799) | (-1.010992169380188, 1.0064207315444946) |
| 743 | (-1.2445138692855835, 1.3402776718139648) | (-1.0057727098464966, 1.0039398670196533) |
| 744 | (-1.3596137762069702, 1.3683218955993652) | (-0.9972085356712341, 0.9974085688591003) |
| 745 | (-1.2471336126327515, 1.404409646987915) | (-1.0027416944503784, 1.0070244073867798) |
| 746 | (-1.3205621242523193, 1.372154712677002) | (-0.9901622533798218, 0.9916868209838867) |
| 747 | (-1.3156931400299072, 1.2873479127883911) | (-1.0097291571034822, 1.00381718941043) |
| 748 | (-1.3299261331558228, 1.317609190940857) | (-0.8797667622566223, 1.0754873752593994) |
| 749 | (-1.5017642974853516, 1.4287903308868408) | (-1.0074468851089478, 1.025960087776184) |
| 750 | (-1.308502197265625, 1.2942687273025513) | (-1.0150307416915894, 1.0098286867141724) |
| 751 | (-1.3380966186523438, 1.3764466047286987) | (-1.002090573310852, 1.0082029104232788) |
| 752 | (-1.2715799808502197, 1.4507906436920166) | (-0.8685380220413208, 1.1087627410888672) |
| 753 | (-1.3522653579711914, 1.3445720672607422) | (-1.0092512369155884, 1.0076923370361328) |
| 754 | (-1.3265689611434937, 1.438887357711792) | (-0.9861285090446472, 1.000508427619934) |
| 755 | (-1.294958472251892, 1.3534362316131592) | (-1.0015480518341064, 1.0023003816604614) |
| 756 | (-1.3384078741073608, 1.3563497066497803) | (-0.9938606023788452, 1.001101016998291) |
| 757 | (-1.4191980361938477, 1.321178913116455) | (-0.9996057748794556, 1.0037062168121338) |
| 758 | (-1.5112900733947754, 1.4011929035186768) | (-0.860217273235321, 1.117073893547058) |
| 759 | (-1.386092185974121, 1.3920974731445312) | (-0.9975230693817139, 1.0080409049987793) |
| 760 | (-1.3043439388275146, 1.2334736585617065) | (-0.9892275333404541, 1.0039576292037964) |
| 761 | (-1.2536977529525757, 1.3531726598739624) | (-1.0059690475463867, 1.0184292793273926) |
| 762 | (-1.4380552768707275, 1.327239990234375) | (-0.9781872630119324, 0.984939455986023) |
| 763 | (-1.3787083625793457, 1.2542232275009155) | (-0.9722471833229065, 0.9805310964584351) |
| 764 | (-1.304305076599121, 1.5538724660873413) | (-1.0016744136810303, 0.9982813596725464) |
| 765 | (-1.3688820600509644, 1.3502520322799683) | (-1.0004465579986572, 1.0156227350234985) |
| 766 | (-1.3390464782714844, 1.3763405084609985) | (-1.0040745735168457, 0.9961997270584106) |
| 767 | (-1.4002138376235962, 1.3922256231307983) | (-0.9866982102394104, 1.0004043579101562) |
| 768 | (-1.3136919736862183, 1.3374595642089844) | (-1.000187873840332, 1.002975583076477) |
| 769 | (-1.3110979795455933, 1.4501994848251343) | (-0.9977476596832275, 1.0101938247680664) |
| 770 | (-1.3970063924789429, 1.3092925548553467) | (-0.9965794086456299, 0.9931493401527405) |
| 771 | (-1.2751442193984985, 1.3813060522079468) | (-1.0000191926956177, 1.0042845010757446) |
| 772 | (-1.238631010055542, 1.2542836666107178) | (-0.9784197807312012, 1.000490427017212) |
| 773 | (-1.406581163406372, 1.4052166938781738) | (-0.9967585206031799, 0.9934943914413452) |
| 774 | (-1.3686397075653076, 1.3371083736419678) | (-0.9924201965332031, 1.0042632818222046) |
| 775 | (-1.2682998180389404, 1.2048189640045166) | (-0.9950354099273682, 0.9879001975059509) |
| 776 | (-1.3264906406402588, 1.287537693977356) | (-1.001976490020752, 1.0024138689041138) |
| 777 | (-1.3531041145324707, 1.2691237926483154) | (-0.9863085746765137, 1.0001081228256226) |
| 778 | (-1.3484325408935547, 1.2841782569885254) | (-1.0038403272628784, 1.0048316717147827) |
| 779 | (-1.3570367097854614, 1.3765406608581543) | (-1.0097261571034822, 1.00381718941643) |
| 780 | (-1.3836584091186523, 1.4820674657821655) | (-0.9931778907775879, 0.9987626075744629) |
| 781 | (-1.3133001327514648, 1.2385808229446411) | (-0.9924511909484863, 0.9809277653694153) |
| 782 | (-1.3396093845367432, 1.4462344646453857) | (-1.0006129741668701, 1.0109351873397827) |
| 783 | (-1.3193655014038086, 1.318451166152954) | (-0.9957561492919922, 1.0043079853057861) |
| 784 | (-1.3176275491714478, 1.2767081260681152) | (-0.9861463904380798, 0.9924638867378235) |
| 785 | (-1.3255367279052734, 1.3701847791671753) | (-0.8902698755264282, 1.051737666130066) |
| 786 | (-1.3566639423370361, 1.32095468044281) | (-0.97369384765625, 1.0381834506988525) |
| 787 | (-1.4185459613800049, 1.3740975856781006) | (-1.0049774646759033, 1.0053595304489136) |
| 788 | (-1.324759840965271, 1.3447827100753784) | (-0.9919918775558472, 0.9902777671813965) |
| 789 | (-1.275259256362915, 1.320138931274414) | (-0.9562289714813232, 1.057766079902649) |
| 790 | (-1.3584628105163574, 1.3869297504425049) | (-0.9920087456703186, 0.9938063025474548) |
| 791 | (-1.4179831743240356, 1.4751445055007935) | (-1.0033873319625854, 1.0040241479873657) |
| 792 | (-1.3993299007415771, 1.3204455375671387) | (-1.0008589029312134, 0.9824337959289551) |
| 793 | (-1.3042887449264526, 1.4768779277801514) | (-0.9764647483825684, 0.9997871518135071) |
| 794 | (-1.3865902423858643, 1.2608102560043335) | (-1.0009709596633911, 0.9965084195137024) |
| 795 | (-1.308494210243225, 1.306381106376648) | (-1.0179688930511475, 1.0114201307296753) |
| 796 | (-1.3091439008712769, 1.3049378395080566) | (-1.0034253597259521, 0.9982925653457642) |
| 797 | (-1.3516416549682617, 1.3807787895202637) | (-1.0097261575034822, 1.00381718941653) |
| 798 | (-1.3216884136199951, 1.435469627380371) | (-1.0192970037460327, 0.9953768253326416) |
| 799 | (-1.3151099681854248, 1.376772403717041) | (-1.008467435836792, 1.009957194328308) |
| 800 | (-1.3808307647705078, 1.5525224208831787) | (-0.9914349913597107, 1.0139946937561035) |
| 801 | (-1.2555630207061768, 1.4553847312927246) | (-0.9885666370391846, 0.9959331750869751) |
| 802 | (-1.602351427078247, 1.3858489990234375) | (-0.9868035912513733, 1.0069037675857544) |
| 803 | (-1.2786970138549805, 1.254238247871399) | (-0.9804839491844177, 0.9927824139595032) |
| 804 | (-1.3220173120498657, 1.4043062925338745) | (-1.0075496435165405, 0.9917178750038147) |
| 805 | (-1.302640438079834, 1.3020392656326294) | (-0.9999545812606812, 1.0084961652755737) |
| 806 | (-1.2809046506881714, 1.4136533737182617) | (-0.9833013415336609, 0.9836667776107788) |
| 807 | (-1.2767010927200317, 1.3268122673034668) | (-1.0064398050308228, 1.005905270576477) |
| 808 | (-1.3439985513687134, 1.4062414169311523) | (-0.9906200766563416, 1.0020015239715576) |
| 809 | (-1.3703676462173462, 1.3450396060943604) | (-0.9784971475601196, 0.997527003288269) |
| 810 | (-1.3108900785446167, 1.3225735425949097) | (-0.9965741634368896, 0.9958041906356812) |
| 811 | (-1.407856822013855, 1.5144789218902588) | (-1.0055526494979858, 0.9906813502311707) |
| 812 | (-1.439913272857666, 1.3847401142120361) | (-1.012434959411621, 1.0102777481079102) |
| 813 | (-1.3576593399047852, 1.3596282005310059) | (-1.00993013381958, 1.0069622993469238) |
| 814 | (-1.3330061435699463, 1.3669012784957886) | (-1.0216552019119263, 0.9960179328918457) |
| 815 | (-1.360629916191101, 1.3348743915557861) | (-1.0255343914031982, 1.0106792449951172) |
| 816 | (-1.4458074569702148, 1.3602784872055054) | (-0.9931924939155579, 1.0008114576339722) |
| 817 | (-1.3141200542449951, 1.4166984558105469) | (-0.9955736398696899, 0.9858747124671936) |
| 818 | (-1.3665692806243896, 1.3057458400726318) | (-1.0126649141311646, 1.0087052583694458) |
| 819 | (-1.3433640003204346, 1.2874201536178589) | (-1.0076444149017334, 0.9919917583465576) |
| 820 | (-1.3450243473052979, 1.266398310661316) | (-0.9988524913787842, 0.9662288427352905) |
| 821 | (-1.387515664100647, 1.3989787101745605) | (-0.9968046545982361, 1.0052905082702637) |
| 822 | (-1.3928351402282715, 1.3742923736572266) | (-1.0046888589859009, 1.010380506515503) |
| 823 | (-1.3744007349014282, 1.2995941638946533) | (-0.9907217025756836, 0.9875084757804871) |
| 824 | (-1.2322062253952026, 1.435566782951355) | (-1.014029860496521, 1.0042824745178223) |
| 825 | (-1.3729519844055176, 1.4239232540130615) | (-0.9840096831321716, 0.9973703026771545) |
| 826 | (-1.4101518392562866, 1.2693146467208862) | (-0.9887683391571045, 0.996190071105957) |
| 827 | (-1.2918399572372437, 1.3470338582992554) | (-1.007887363433838, 0.996537446975708) |
| 828 | (-1.264197826385498, 1.3118247985839844) | (-0.9854604601860046, 0.9918253421783447) |
| 829 | (-1.3631807565689087, 1.2306045293807983) | (-1.009726157544822, 1.00381418941653) |
| 830 | (-1.3791546821594238, 1.4503247737884521) | (-0.9958693981170654, 0.9835332632064819) |
| 831 | (-1.3584239482879639, 1.5230703353881836) | (-0.9889101386070251, 0.9912538528442383) |
| 832 | (-1.4856799840927124, 1.4617195129394531) | (-1.0061390399932861, 0.9981185793876648) |
| 833 | (-1.4184074401855469, 1.3327053785324097) | (-1.0162379741668701, 1.0085142850875854) |
| 834 | (-1.2490452527999878, 1.3160756826400757) | (-1.0075119733810425, 1.002313256263733) |
| 835 | (-1.3474797010421753, 1.3381171226501465) | (-1.0088222026824951, 1.002701759338379) |
| 836 | (-1.3514671325683594, 1.3504807949066162) | (-0.9936382174491882, 0.9970217347145081) |
| 837 | (-1.3789087533950806, 1.4370176792144775) | (-1.004538893699646, 0.998013436794281) |
| 838 | (-1.316468358039856, 1.308072805404663) | (-1.007758378982544, 1.0248655080795288) |
| 839 | (-1.3493348360061646, 1.2865806818008423) | (-1.0004324913024902, 0.9985239505767822) |
| 840 | (-1.3234975337982178, 1.400771141052246) | (-1.001926302909851, 1.002768635749817) |
| 841 | (-1.458923101425171, 1.3834675550460815) | (-1.0069942474365234, 0.9880930185317993) |
| 842 | (-1.335966944694519, 1.2456746101379395) | (-1.0047831535339355, 1.0031672716140747) |
| 843 | (-1.3532460927963257, 1.3743550777435303) | (-1.0023466348648071, 1.0077935457229614) |
| 844 | (-1.347454309463501, 1.3997513055801392) | (-1.0076980590820312, 1.0024173259735107) |
| 845 | (-1.4084348678588867, 1.2526947259902954) | (-0.9700628519058228, 1.0190812349319458) |
| 846 | (-1.352393627166748, 1.3341422080993652) | (-0.9691934585571289, 0.9692056775093079) |
| 847 | (-1.3870315551757812, 1.3287482261657715) | (-0.9800991415977478, 1.0156395435333252) |
| 848 | (-1.35464608669281, 1.3355438709259033) | (-1.0077828168869019, 1.0023378133773804) |
| 849 | (-1.335971713066101, 1.4447603225708008) | (-1.0158488750457764, 1.001856803894043) |
| 850 | (-1.3005365133285522, 1.4379582405090332) | (-0.9844748377799988, 0.9931859970092773) |
| 851 | (-1.3057810068130493, 1.2759733200073242) | (-0.9964299201965332, 1.0099302530288696) |
| 852 | (-1.419267177581787, 1.30755615234375) | (-1.0116794109344482, 1.0143101215362549) |
| 853 | (-1.3653308153152466, 1.3189353942871094) | (-0.9964824914932251, 1.0190939903259277) |
| 854 | (-1.3544912338256836, 1.3804055452346802) | (-1.0058988332748413, 0.9952120780944824) |
| 855 | (-1.3620665073394775, 1.2355214357376099) | (-1.003699541091919, 1.0096356868743896) |
| 856 | (-1.2872836589813232, 1.3583338260650635) | (-1.0005146265029907, 1.0014824867248535) |
| 857 | (-1.3949687480926514, 1.3352516889572144) | (-0.9809538125991821, 1.0198125839233398) |
| 858 | (-1.2999467849731445, 1.4040783643722534) | (-0.915507435798645, 1.0643219947814941) |
| 859 | (-1.3393216133117676, 1.2773782014846802) | (-1.015136480331421, 0.9899997115135193) |
| 860 | (-1.278973937034607, 1.3747172355651855) | (-1.080597996711731, 0.9680314660072327) |
| 861 | (-1.3331804275512695, 1.3467156887054443) | (-1.0047565698623657, 0.9944204688072205) |
| 862 | (-1.3664867877960205, 1.3704595565795898) | (-0.9898129105567932, 1.0122405290603638) |
| 863 | (-1.3061174154281616, 1.3242790699005127) | (-0.9921249151229858, 1.0003916025161743) |
| 864 | (-1.398468255996704, 1.3206042051315308) | (-1.0049992799758911, 1.0031743049621582) |
| 865 | (-1.4250786304473877, 1.312754511833191) | (-0.9964576959609985, 0.989029586315155) |
| 866 | (-1.379731297492981, 1.3156136274337769) | (-1.0151108503341675, 0.9933103919029236) |
| 867 | (-1.4199916124343872, 1.2829606533050537) | (-0.9992260336875916, 0.9895102977752686) |
| 868 | (-1.3623943328857422, 1.5130300521850586) | (-0.9951181411743164, 0.998650312423706) |
| 869 | (-1.3235375881195068, 1.4174351692199707) | (-0.9844287633895874, 0.9969841241836548) |
| 870 | (-1.429721713066101, 1.3928992748260498) | (-0.9224653244018555, 1.0414692163467407) |
| 871 | (-1.3088812828063965, 1.376208782196045) | (-1.0106146335601807, 1.001926064491272) |
| 872 | (-1.3558342456817627, 1.3391704559326172) | (-1.01692795753479, 0.9955505728721619) |
| 873 | (-1.4534531831741333, 1.3574901819229126) | (-0.9931173324584961, 0.9985717535018921) |
| 874 | (-1.2442134618759155, 1.370718240737915) | (-0.9907602667808533, 1.00861656665802) |
| 875 | (-1.3937187194824219, 1.3259108066558838) | (-1.007012963294983, 0.9937952756881714) |
| 876 | (-1.3117632865905762, 1.2793666124343872) | (-0.9839069843292236, 1.0030691623687744) |
| 877 | (-1.2543482780456543, 1.495315432548523) | (-0.9958789348602295, 1.0082170963287354) |
| 878 | (-1.3803982734680176, 1.358445644378662) | (-1.008829116821289, 1.0062388181686401) |
| 879 | (-1.3711398839950562, 1.3691215515136719) | (-0.9890518188476562, 0.987592875957489) |
| 880 | (-1.3540961742401123, 1.3551992177963257) | (-1.005721092224121, 0.9889232516288757) |
| 881 | (-1.3218624591827393, 1.4299126863479614) | (-1.0014909505844116, 1.004300594329834) |
| 882 | (-1.3817451000213623, 1.4356014728546143) | (-1.0028811693191528, 0.993346095085144) |
| 883 | (-1.3197696208953857, 1.4363880157470703) | (-1.012474536895752, 0.9993879199028015) |
| 884 | (-1.325408697128296, 1.3405163288116455) | (-0.9966930747032166, 0.9968172907829285) |
| 885 | (-1.3006548881530762, 1.3189736604690552) | (-0.9843525290489197, 0.9999595880508423) |
| 886 | (-1.3675734996795654, 1.3855470418930054) | (-0.9850860238075256, 1.0284404754638672) |
| 887 | (-1.2732470035552979, 1.3808780908584595) | (-1.0011005401611328, 1.012088418006897) |
| 888 | (-1.277834415435791, 1.4408072233200073) | (-0.9988361597061157, 1.0029418468475342) |
| 889 | (-1.3968294858932495, 1.2873382568359375) | (-0.9414546489715576, 1.009440302848816) |
| 890 | (-1.303865909576416, 1.4009302854537964) | (-0.994185209274292, 1.008777379989624) |
| 891 | (-1.3890196084976196, 1.445638656616211) | (-1.0067088603973389, 0.9894545078277588) |
| 892 | (-1.331050157546997, 1.3449379205703735) | (-0.9955801963806152, 0.9942565560340881) |
| 893 | (-1.3626766204833984, 1.3475335836410522) | (-0.9893178343772888, 0.9890323281288147) |
| 894 | (-1.3193941116333008, 1.3458703756332397) | (-1.0084880590438843, 1.0013344287872314) |
| 895 | (-1.3851664066314697, 1.2673985958099365) | (-0.9818684458732605, 0.9905243515968323) |
| 896 | (-1.4217994213104248, 1.3165228366851807) | (-1.0044732093811035, 1.0013233423233032) |
| 897 | (-1.3634727001190186, 1.37709641456604) | (-1.0087172985076904, 0.993847131729126) |
| 898 | (-1.3356679677963257, 1.3051108121871948) | (-0.9984759092330933, 1.0010696649551392) |
| 899 | (-1.3514916896820068, 1.3025916814804077) | (-0.9428526759147644, 0.9827942252159119) |
| 900 | (-1.3538061380386353, 1.294796347618103) | (-1.0034618377685547, 0.9980005025863647) |
| 901 | (-1.3927947282791138, 1.3186277151107788) | (-1.009722157544822, 1.00381418941253) |
| 902 | (-1.3860113620758057, 1.3769317865371704) | (-1.005983829498291, 0.9934279918670654) |
| 903 | (-1.2505441904067993, 1.322533369064331) | (-0.9845232963562012, 1.0248463153839111) |
| 904 | (-1.3583966493606567, 1.415461778640747) | (-1.0161452293395996, 1.0088402032852173) |
| 905 | (-1.316583514213562, 1.2494711875915527) | (-1.0063469409942627, 0.9935526847839355) |
| 906 | (-1.3624932765960693, 1.2776894569396973) | (-1.0045346021652222, 1.0010594129562378) |
| 907 | (-1.394670009613037, 1.344597339630127) | (-0.9981716871261597, 1.0162758827209473) |
| 908 | (-1.331845998764038, 1.3699380159378052) | (-1.003555417060852, 1.0085203647613525) |
| 909 | (-1.3203775882720947, 1.3221875429153442) | (-1.013110876083374, 1.0142987966537476) |
| 910 | (-1.3501086235046387, 1.3733017444610596) | (-0.9419153332710266, 1.0203490257263184) |
| 911 | (-1.4300365447998047, 1.435448408126831) | (-0.9987057447433472, 0.9868481159210205) |
| 912 | (-1.3658379316329956, 1.2815041542053223) | (-0.9873478412628174, 0.9860371947288513) |
| 913 | (-1.4255495071411133, 1.3619402647018433) | (-0.9903773069381714, 0.9892271757125854) |
| 914 | (-1.2781927585601807, 1.3903002738952637) | (-0.9935619235038757, 1.026674747467041) |
| 915 | (-1.4372636079788208, 1.2819764614105225) | (-0.9939444065093994, 0.9977935552597046) |
| 916 | (-1.300528883934021, 1.3910434246063232) | (-1.009722151544822, 1.00381418911253) |
| 917 | (-1.2918510437011719, 1.3633980751037598) | (-0.9506995677947998, 1.0528680086135864) |
| 918 | (-1.2958333492279053, 1.3260698318481445) | (-0.9917700290679932, 0.983439564704895) |
| 919 | (-1.270804762840271, 1.2928707599639893) | (-0.9929060935974121, 1.0092114210128784) |
| 920 | (-1.3069614171981812, 1.4112666845321655) | (-1.0062577724456787, 0.9962046146392822) |
| 921 | (-1.414946436882019, 1.3392966985702515) | (-0.9967013001441956, 1.0036468505859375) |
| 922 | (-1.333292007446289, 1.3442366123199463) | (-1.0156899690628052, 1.0036910772323608) |
| 923 | (-1.3192998170852661, 1.3369158506393433) | (-1.0148961544036865, 1.0136700868606567) |
| 924 | (-1.369604229927063, 1.3945434093475342) | (-1.0047036409378052, 0.9875224828720093) |
| 925 | (-1.3167760372161865, 1.4169435501098633) | (-1.0045911073684692, 1.004067063331604) |
| 926 | (-1.3185068368911743, 1.4061529636383057) | (-0.9926793575286865, 0.9997421503067017) |
| 927 | (-1.371520757675171, 1.3498311042785645) | (-1.009722151545822, 1.00381418511253) |
| 928 | (-1.3048741817474365, 1.3575589656829834) | (-1.0017964839935303, 0.9964191317558289) |
| 929 | (-1.329785943031311, 1.5474350452423096) | (-0.9943627715110779, 1.0003745555877686) |
| 930 | (-1.3465774059295654, 1.3193408250808716) | (-1.008420467376709, 0.9944449067115784) |
| 931 | (-1.2825380563735962, 1.3841133117675781) | (-1.0224171876907349, 1.0001131296157837) |
| 932 | (-1.3528231382369995, 1.252953052520752) | (-1.0020891427993774, 0.9949972033500671) |
| 933 | (-1.2902153730392456, 1.4395166635513306) | (-1.003764271736145, 1.0046992301940918) |
| 934 | (-1.3186297416687012, 1.309757947921753) | (-1.0027471780776978, 1.0124588012695312) |
| 935 | (-1.39459228515625, 1.3422014713287354) | (-1.009722751545822, 1.00381478511253) |
| 936 | (-1.303621768951416, 1.3292800188064575) | (-1.006348967552185, 1.0073741674423218) |
| 937 | (-1.3566009998321533, 1.3042571544647217) | (-0.9929754137992859, 0.9870069026947021) |
| 938 | (-1.3326770067214966, 1.3384827375411987) | (-0.9989086985588074, 0.9978128671646118) |
| 939 | (-1.2502330541610718, 1.3513085842132568) | (-1.0064841508865356, 1.0026440620422363) |
| 940 | (-1.3465678691864014, 1.3901116847991943) | (-1.0089735984802246, 1.0067548751831055) |
| 941 | (-1.3270267248153687, 1.338070273399353) | (-1.0146832466125488, 1.0055382251739502) |
| 942 | (-1.2862181663513184, 1.454627275466919) | (-0.9799191951751709, 0.999273955821991) |
| 943 | (-1.3896021842956543, 1.3480708599090576) | (-1.0019854307174683, 1.0113078355789185) |
| 944 | (-1.3662052154541016, 1.3628668785095215) | (-1.0142170190811157, 1.0055255889892578) |
| 945 | (-1.3600897789001465, 1.3739107847213745) | (-0.9833377003669739, 1.0045758485794067) |
| 946 | (-1.4796910285949707, 1.4930918216705322) | (-0.9868033528327942, 0.9843999743461609) |
| 947 | (-1.411260724067688, 1.2640682458877563) | (-0.9923994541168213, 1.0070990324020386) |
| 948 | (-1.3419262170791626, 1.3574846982955933) | (-1.0000742673873901, 1.0040972232818604) |
| 949 | (-1.2664316892623901, 1.2917933464050293) | (-1.0075562000274658, 0.9911025762557983) |
| 950 | (-1.33403480052948, 1.3049132823944092) | (-0.996917724609375, 0.9902830123901367) |
| 951 | (-1.3557847738265991, 1.3894305229187012) | (-0.9882232546806335, 1.0124834775924683) |
| 952 | (-1.402044653892517, 1.3929901123046875) | (-0.9223895072937012, 1.0015995502471924) |
| 953 | (-1.3080220222473145, 1.3987053632736206) | (-0.9968781471252441, 0.9956316947937012) |
| 954 | (-1.370906114578247, 1.2554137706756592) | (-0.9964296817779541, 0.9849122762680054) |
| 955 | (-1.4172618389129639, 1.3070290088653564) | (-1.0127785205841064, 1.0078279972076416) |
| 956 | (-1.3148410320281982, 1.3104510307312012) | (-0.9762926697731018, 0.9945298433303833) |
| 957 | (-1.3484617471694946, 1.4245681762695312) | (-0.9978659152984619, 0.9972833395004272) |
| 958 | (-1.3540291786193848, 1.422435998916626) | (-1.0094724893569946, 1.015121340751648) |
| 959 | (-1.3512353897094727, 1.29876708984375) | (-0.9813752174377441, 0.9856635928153992) |
| 960 | (-1.337686538696289, 1.3493455648422241) | (-1.0044825077056885, 0.9976261854171753) |
| 961 | (-1.3100134134292603, 1.3211266994476318) | (-1.0186378955841064, 1.0109044313430786) |
| 962 | (-1.4186804294586182, 1.2736753225326538) | (-1.016853928565979, 1.0018253326416016) |
| 963 | (-1.2884658575057983, 1.3672759532928467) | (-0.9992930889129639, 1.0109717845916748) |
| 964 | (-1.2888307571411133, 1.3603733777999878) | (-0.9824804663658142, 0.9949767589569092) |
| 965 | (-1.2849289178848267, 1.285738468170166) | (-1.009722751525822, 1.00381478511223) |
| 966 | (-1.2740105390548706, 1.3397085666656494) | (-0.96022629737854, 1.0343308448791504) |
| 967 | (-1.3611507415771484, 1.3520002365112305) | (-1.0055184364318848, 0.99957275390625) |
| 968 | (-1.4318726062774658, 1.4419293403625488) | (-0.9975501894950867, 0.99115389585495) |
| 969 | (-1.369563341140747, 1.2997148036956787) | (-1.0041860342025757, 0.9801831841468811) |
| 970 | (-1.3391599655151367, 1.3189961910247803) | (-1.0040727853775024, 1.0030136108398438) |
| 971 | (-1.322235345840454, 1.3263658285140991) | (-1.009722751825822, 1.00381478518223) |
| 972 | (-1.328163743019104, 1.3350765705108643) | (-1.0145050287246704, 1.009695053100586) |
| 973 | (-1.4169244766235352, 1.279234528541565) | (-0.9027343392372131, 1.0826034545898438) |
| 974 | (-1.3294256925582886, 1.2886004447937012) | (-0.9927371144294739, 1.0000303983688354) |
| 975 | (-1.31280517578125, 1.3076562881469727) | (-0.9903451800346375, 0.9856181144714355) |
| 976 | (-1.2734893560409546, 1.3139785528182983) | (-1.0019465684890747, 1.0112930536270142) |
| 977 | (-1.2625727653503418, 1.4253627061843872) | (-0.9887986183166504, 1.0218138694763184) |
| 978 | (-1.3334758281707764, 1.4559779167175293) | (-0.9816850423812866, 1.020074725151062) |
| 979 | (-1.278381109237671, 1.4419817924499512) | (-0.9814379215240479, 0.9913730621337891) |
| 980 | (-1.2069621086120605, 1.3574559688568115) | (-1.003960371017456, 1.0100274085998535) |
| 981 | (-1.3914607763290405, 1.3988085985183716) | (-1.003252625465393, 1.0040318965911865) |
| 982 | (-1.3652865886688232, 1.3347201347351074) | (-0.9921461939811707, 0.9987400770187378) |
| 983 | (-1.3619698286056519, 1.3748018741607666) | (-0.9884364604949951, 1.0133236646652222) |
| 984 | (-1.3028793334960938, 1.354979157447815) | (-1.000684142112732, 1.0012930631637573) |
| 985 | (-1.2850985527038574, 1.336202621459961) | (-0.9837213754653931, 0.9926992654800415) |
| 986 | (-1.2983553409576416, 1.2892649173736572) | (-1.009722791825822, 1.00381478519223) |
| 987 | (-1.3954339027404785, 1.3562361001968384) | (-0.9982749819755554, 1.0101217031478882) |
| 988 | (-1.3466525077819824, 1.3422815799713135) | (-1.0017446279525757, 1.002099871635437) |
| 989 | (-1.3022270202636719, 1.3743199110031128) | (-0.995935320854187, 0.9907318353652954) |
| 990 | (-1.3944863080978394, 1.3483920097351074) | (-0.9969890117645264, 1.0012140274047852) |
| 991 | (-1.296203851699829, 1.3889985084533691) | (-1.0082294940948486, 0.9967623353004456) |
| 992 | (-1.3338654041290283, 1.28304922580719) | (-0.9945603609085083, 0.9978874325752258) |
| 993 | (-1.3970146179199219, 1.3841303586959839) | (-0.9965508580207825, 0.9869274497032166) |
| 994 | (-1.4443578720092773, 1.3013356924057007) | (-0.9922187328338623, 1.0005066394805908) |
| 995 | (-1.3644081354141235, 1.3324767351150513) | (-0.985880434513092, 0.9993789196014404) |
| 996 | (-1.375646710395813, 1.4770642518997192) | (-0.9991954565048218, 0.989777684211731) |
| 997 | (-1.2799044847488403, 1.3846912384033203) | (-0.992074191570282, 0.9968262314796448) |
| 998 | (-1.2270214557647705, 1.4276543855667114) | (-1.0047959089279175, 0.9999666213989258) |
| 999 | (-1.3188252449035645, 1.3454965353012085) | (-1.0043638944625854, 0.9978678822517395) |
| 1000 | (-1.3790265321731567, 1.347108244895935) | (-0.9992809891700745, 0.9995303750038147) |
"""

bounding_variance = 2*variance
parsed_data = parse_table(text)
print("Data Statistics:")
data_statistics = performance_statistics(parsed_data, "Data Codomain", mean, bounding_variance, (-1, 1))

print("KAN Statistics:")
kan_statistics = performance_statistics(parsed_data, "KAN Codomain", mean, bounding_variance, (-1, 1))


Data Statistics:


,Real Codomain,Mean Codomain,Mean Deviation,Max Deviation,Min Deviation,Bounding Variance,% Below Variance
X,-1,-1.345510,0.345510,0.602351,0.206962,-0.04,0.0
Y,1,1.352241,0.352241,0.599259,0.204819,0.04,0.0


KAN Statistics:


,Real Codomain,Mean Codomain,Mean Deviation,Max Deviation,Min Deviation,Bounding Variance,% Below Variance
X,-1,-0.994404,0.012855,0.146107,0.000019,-0.04,93.9
Y,1,1.002331,0.010506,0.146250,0.000030,0.04,95.7
